In [1]:
import os
import gc
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.utils import resample
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Configuration
folds = range(0, 5)
stacking_path = '/UK_BB/brainbody/stacking'
commonality_path = '/UK_BB/brainbody/commonality'
os.makedirs(commonality_path, exist_ok=True)

In [ ]:
# Define helper functions 
def commonality_analysis(features_g_pred, target_g_real):
    model = LinearRegression()
    if len(features_g_pred.shape) < 2:
        features = features_g_pred.values.reshape(-1, 1)
    else:
        features = features_g_pred
    model.fit(features, target_g_real)
    return round(model.score(features, target_g_real), 4)

def decompose_variance(row):
    """Calculate variance components for a single row"""
    u_brain = row['R2_combined'] - row['R2_body']
    u_body = row['R2_combined'] - row['R2_brain']
    common = row['R2_brain'] + row['R2_body'] - row['R2_combined']
    
    return {
        'Unique variance: brain': round(u_brain, 4),
        'Unique variance: body': round(u_body, 4),
        'Common variance': round(common, 4),
        'Perc unique brain': round((u_brain/row['R2_combined'])*100, 2) if row['R2_combined'] else 0,
        'Perc unique body': round((u_body/row['R2_combined'])*100, 2) if row['R2_combined'] else 0,
        'Perc common': round((common/row['R2_combined'])*100, 2) if row['R2_combined'] else 0,
        'g-brain explained by body': round((common/(u_brain+common))*100, 2) if (u_brain+common) else 0,
        'g-body explained by brain': round((common/(u_body+common))*100, 2) if (u_body+common) else 0
    }


In [3]:
# Define brain and body modalities
###############################################################
modalities_brain_body = [
'immune',
'renalhepatic',
'metabolic',
'cardiopulmonary',
'musculoskeletal',
'bone_densitometry',
'pwa',
'heart_mri',
'carotid_ultrasound',
'arterial_stiffness',
'ecg_rest',
'body_composition_by_impedance',
'body_composition_dxa',
'bone_dxa',
'kidneys_mri',
'liver_mri',
'abdominal_composition_mri_18_vars', #17 vars
'abdominal_organ_composition_mri_13_vars', #12 vars
'hearing',

'struct_fast',
'struct_sub_first',
'struct_fs_aseg_mean_intensity',
'struct_fs_aseg_volume',
'struct_ba_exvivo_area', 
'struct_ba_exvivo_mean_thickness',
'struct_ba_exvivo_volume',
'struct_a2009s_area',
'struct_a2009s_mean_thickness',
'struct_a2009s_volume',
'struct_dkt_area',
'struct_dkt_mean_thickness',
'struct_dkt_volume',
'struct_desikan_gw',
'struct_desikan_pial',
'struct_desikan_white_area',
'struct_desikan_white_mean_thickness',
'struct_desikan_white_volume',
'struct_subsegmentation',
'add_t1',
'add_t2',

"dwi_FA_tbss", "dwi_FA_prob",
"dwi_MD_tbss", "dwi_MD_prob",
"dwi_L1_tbss", "dwi_L1_prob",
"dwi_L2_tbss", "dwi_L2_prob",
"dwi_L3_tbss", "dwi_L3_prob",
"dwi_MO_tbss", "dwi_MO_prob",
"dwi_OD_tbss", "dwi_OD_prob",
"dwi_ICVF_tbss", "dwi_ICVF_prob",
"dwi_ISOVF_tbss", "dwi_ISOVF_prob",

'aparc_Tian_S1_FA_i2',
'aparc_Tian_S1_Length_i2',
'aparc_Tian_S1_SIFT2_FBC_i2',
'aparc_Tian_S1_Streamline_Count_i2',

'aparc_a2009s_Tian_S1_FA_i2',
'aparc_a2009s_Tian_S1_Length_i2',
'aparc_a2009s_Tian_S1_SIFT2_FBC_i2',
'aparc_a2009s_Tian_S1_Streamline_Count_i2',

'Glasser_Tian_S1_FA_i2',
'Glasser_Tian_S1_Length_i2',
'Glasser_Tian_S1_SIFT2_FBC_i2',
'Glasser_Tian_S1_Streamline_Count_i2',

'Glasser_Tian_S4_FA_i2',
'Glasser_Tian_S4_Length_i2',
'Glasser_Tian_S4_SIFT2_FBC_i2',
'Glasser_Tian_S4_Streamline_Count_i2',

'Schaefer7n200p_Tian_S1_FA_i2',
'Schaefer7n200p_Tian_S1_Length_i2',
'Schaefer7n200p_Tian_S1_SIFT2_FBC_i2',
'Schaefer7n200p_Tian_S1_Streamline_Count_i2',

'Schaefer7n1000p_Tian_S4_FA_i2',
'Schaefer7n1000p_Tian_S4_Length_i2',
'Schaefer7n1000p_Tian_S4_SIFT2_FBC_i2',
'Schaefer7n1000p_Tian_S4_Streamline_Count_i2',

"amplitudes_21",
"full_correlation_21",
"partial_correlation_21",
"amplitudes_55",
"full_correlation_55",
"partial_correlation_55",
'full_correlation_aparc_a2009s_Tian_S1',
'full_correlation_aparc_Tian_S1',
'full_correlation_Glasser_Tian_S1',
'full_correlation_Glasser_Tian_S4',
'full_correlation_Schaefer7n200p_Tian_S1',
'full_correlation_Schaefer7n500p_Tian_S4',
'partial_correlation_aparc_a2009s_Tian_S1',
'partial_correlation_aparc_Tian_S1',
'partial_correlation_Glasser_Tian_S1',
'partial_correlation_Glasser_Tian_S4',
'partial_correlation_Schaefer7n200p_Tian_S1',
'partial_correlation_Schaefer7n500p_Tian_S4'
]

modalities_brain = [
'struct_fast',
'struct_sub_first',
'struct_fs_aseg_mean_intensity',
'struct_fs_aseg_volume',
'struct_ba_exvivo_area', 
'struct_ba_exvivo_mean_thickness',
'struct_ba_exvivo_volume',
'struct_a2009s_area',
'struct_a2009s_mean_thickness',
'struct_a2009s_volume',
'struct_dkt_area',
'struct_dkt_mean_thickness',
'struct_dkt_volume',
'struct_desikan_gw',
'struct_desikan_pial',
'struct_desikan_white_area',
'struct_desikan_white_mean_thickness',
'struct_desikan_white_volume',
'struct_subsegmentation',
'add_t1',
'add_t2',

"dwi_FA_tbss", "dwi_FA_prob",
"dwi_MD_tbss", "dwi_MD_prob",
"dwi_L1_tbss", "dwi_L1_prob",
"dwi_L2_tbss", "dwi_L2_prob",
"dwi_L3_tbss", "dwi_L3_prob",
"dwi_MO_tbss", "dwi_MO_prob",
"dwi_OD_tbss", "dwi_OD_prob",
"dwi_ICVF_tbss", "dwi_ICVF_prob",
"dwi_ISOVF_tbss", "dwi_ISOVF_prob",

'aparc_Tian_S1_FA_i2',
'aparc_Tian_S1_Length_i2',
'aparc_Tian_S1_SIFT2_FBC_i2',
'aparc_Tian_S1_Streamline_Count_i2',

'aparc_a2009s_Tian_S1_FA_i2',
'aparc_a2009s_Tian_S1_Length_i2',
'aparc_a2009s_Tian_S1_SIFT2_FBC_i2',
'aparc_a2009s_Tian_S1_Streamline_Count_i2',

'Glasser_Tian_S1_FA_i2',
'Glasser_Tian_S1_Length_i2',
'Glasser_Tian_S1_SIFT2_FBC_i2',
'Glasser_Tian_S1_Streamline_Count_i2',

'Glasser_Tian_S4_FA_i2',
'Glasser_Tian_S4_Length_i2',
'Glasser_Tian_S4_SIFT2_FBC_i2',
'Glasser_Tian_S4_Streamline_Count_i2',

'Schaefer7n200p_Tian_S1_FA_i2',
'Schaefer7n200p_Tian_S1_Length_i2',
'Schaefer7n200p_Tian_S1_SIFT2_FBC_i2',
'Schaefer7n200p_Tian_S1_Streamline_Count_i2',

'Schaefer7n1000p_Tian_S4_FA_i2',
'Schaefer7n1000p_Tian_S4_Length_i2',
'Schaefer7n1000p_Tian_S4_SIFT2_FBC_i2',
'Schaefer7n1000p_Tian_S4_Streamline_Count_i2',

"amplitudes_21",
"full_correlation_21",
"partial_correlation_21",
"amplitudes_55",
"full_correlation_55",
"partial_correlation_55",
'full_correlation_aparc_a2009s_Tian_S1',
'full_correlation_aparc_Tian_S1',
'full_correlation_Glasser_Tian_S1',
'full_correlation_Glasser_Tian_S4',
'full_correlation_Schaefer7n200p_Tian_S1',
'full_correlation_Schaefer7n500p_Tian_S4',
'partial_correlation_aparc_a2009s_Tian_S1',
'partial_correlation_aparc_Tian_S1',
'partial_correlation_Glasser_Tian_S1',
'partial_correlation_Glasser_Tian_S4',
'partial_correlation_Schaefer7n200p_Tian_S1',
'partial_correlation_Schaefer7n500p_Tian_S4'
]
modalities_body = [
'immune',
'renalhepatic',
'metabolic',
'cardiopulmonary',
'musculoskeletal',
'bone_densitometry',
'pwa',
'heart_mri',
'carotid_ultrasound',
'arterial_stiffness',
'ecg_rest',
'body_composition_by_impedance',
'body_composition_dxa',
'bone_dxa',
'kidneys_mri',
'liver_mri',
'abdominal_composition_mri_18_vars', #17 vars
'abdominal_organ_composition_mri_13_vars', #12 vars
'hearing'
]


In [ ]:
# Define modality renaming dictionary (modality_map)
modality_names = {
'hearing': 'Hearing',
'immune': 'Immune',
'renalhepatic': 'Renal & Hepatic',
'metabolic': 'Metabolic',
'cardiopulmonary': 'Cardiopulmonary',
'musculoskeletal': 'Musculoskeletal',
'bone_densitometry': 'Bone Densitometry of Heel',
'pwa': 'Pulse Wave Analysis',
'heart_mri': 'Heart MRI',
'carotid_ultrasound': 'Carotid Ultrasound',
'arterial_stiffness': 'Arterial Stiffness',
'ecg_rest': 'ECG at Rest',
'body_composition_by_impedance': 'Body Composition by Impedance',
'body_composition_dxa': 'Body composition by DXA',
'bone_dxa': 'Bone size, mineral and density by DXA',
'kidneys_mri': 'Kidney MRI',
'liver_mri': 'Liver MRI',
'abdominal_composition_mri_18_vars': 'Abdominal Composition by MRI',
'abdominal_organ_composition_mri_13_vars': 'Abdominal Organ Composition by MRI',
'struct_fast' : 'Regional grey matter volumes (FSL FAST)',
'struct_sub_first': 'Subcortical volumes (FSL FIRST)',

'struct_fs_aseg_mean_intensity' : 'ASEG Mean Intensity',
'struct_fs_aseg_volume' : 'ASEG Volume',


'struct_ba_exvivo_area' : 'BA ex-vivo Area',
'struct_ba_exvivo_mean_thickness' : 'BA ex-vivo Mean Thickness',
'struct_ba_exvivo_volume' : 'BA ex-vivo Volume',

'struct_a2009s_area' : 'a2009s Area',
'struct_a2009s_mean_thickness' : 'a2009s Mean Thickness',
'struct_a2009s_volume' : 'a2009s Volume',


'struct_dkt_area' : 'Desikan-Killiany-Tourville Area',
'struct_dkt_mean_thickness' : 'Desikan-Killiany-Tourville Mean Thickness',
'struct_dkt_volume' : 'Desikan-Killiany-Tourville Volume',


'struct_desikan_gw' : 'Desikan Grey/White Matter Contrast',
'struct_desikan_pial' : 'Desikan Pial',

'struct_desikan_white_area' : 'Desikan White Matter Area',
'struct_desikan_white_mean_thickness' : 'Desikan White Matter Mean Thickness',
'struct_desikan_white_volume' : 'Desikan White Matter Volume',
"struct_subsegmentation":'Subcortical Volumetric Subsegmentation',

'add_t1' : 'Whole-brain T1w',
'add_t2' : 'Whole-brain T2w',
"dwi_FA_tbss": "FA TBSS",
"dwi_FA_prob": "FA Prob.",
"dwi_MD_tbss": "MD TBSS",
"dwi_MD_prob": "MD Prob.",
"dwi_L1_tbss": "L1 TBSS",
"dwi_L1_prob": "L1 Prob.",
"dwi_L2_tbss": "L2 TBSS",
"dwi_L2_prob": "L2 Prob.",
"dwi_L3_tbss": "L3 TBSS",
"dwi_L3_prob": "L3 Prob.",
"dwi_MO_tbss": "MO TBSS",
"dwi_MO_prob": "MO Prob.",
"dwi_OD_tbss": "OD TBSS",
"dwi_OD_prob": "OD Prob.",
"dwi_ICVF_tbss": "ICVF TBSS",
"dwi_ICVF_prob": "ICVF Prob.",
"dwi_ISOVF_tbss": "ISOVF TBSS",
"dwi_ISOVF_prob": "ISOVF Prob.",
"amplitudes_21": " 21 IC amplitudes",
"amplitudes_55": "55 IC amplitudes",
"full_correlation_21": "21 IC Full corr.",
"full_correlation_55": "55 IC Full corr.",
"partial_correlation_21": " 21 IC Partial Corr.",
"partial_correlation_55": " 55 IC Partial Corr.",
# aparc Tian S1 (I)
'aparc_Tian_S1_FA_i2': 'aparc-I FA',
'aparc_Tian_S1_Length_i2': 'aparc-I Length',
'aparc_Tian_S1_SIFT2_FBC_i2': 'aparc-I SIFT2 FBC',
'aparc_Tian_S1_Streamline_Count_i2': 'aparc-I Streamline Count',

# aparc a2009s Tian S1 (I)
'aparc_a2009s_Tian_S1_FA_i2': 'aparc.a2009s-I FA',
'aparc_a2009s_Tian_S1_Length_i2': 'aparc.a2009s-I Length',
'aparc_a2009s_Tian_S1_SIFT2_FBC_i2': 'aparc.a2009s-I SIFT2 FBC',
'aparc_a2009s_Tian_S1_Streamline_Count_i2': 'aparc.a2009s-I Streamline Count',

# Glasser Tian S1 (I)
'Glasser_Tian_S1_FA_i2': 'Glasser-I FA',
'Glasser_Tian_S1_Length_i2': 'Glasser-I Length',
'Glasser_Tian_S1_SIFT2_FBC_i2': 'Glasser-I SIFT2 FBC',
'Glasser_Tian_S1_Streamline_Count_i2': 'Glasser-I Streamline Count',

# Glasser Tian S4 (IV)
'Glasser_Tian_S4_FA_i2': 'Glasser-IV FA',
'Glasser_Tian_S4_Length_i2': 'Glasser-IV Length',
'Glasser_Tian_S4_SIFT2_FBC_i2': 'Glasser-IV SIFT2 FBC',
'Glasser_Tian_S4_Streamline_Count_i2': 'Glasser-IV Streamline Count',

# Schaefer7n1000p Tian S4 (IV) (in reality: Schaefer7n200p Tian S1)
'Schaefer7n1000p_Tian_S4_FA_i2': 'Schaefer7n200p-I FA', #'Schaefer7n1000p-IV FA',
'Schaefer7n1000p_Tian_S4_Length_i2': 'Schaefer7n200p-I Length',#'Schaefer7n1000p-IV Length',
'Schaefer7n1000p_Tian_S4_SIFT2_FBC_i2': 'Schaefer7n200p-I SIFT2 FBC',#'Schaefer7n1000p-IV SIFT2 FBC',
'Schaefer7n1000p_Tian_S4_Streamline_Count_i2': 'Schaefer7n200p-I Streamline Count', #'Schaefer7n1000p-IV Streamline Count'

# Schaefer7n200p Tian S4 (IV) (in reality: Schaefer7n500p Tian S4)
'Schaefer7n200p_Tian_S1_FA_i2': 'Schaefer7n500p-IV FA',
'Schaefer7n200p_Tian_S1_Length_i2': 'Schaefer7n500p-IV Length',
'Schaefer7n200p_Tian_S1_SIFT2_FBC_i2': 'Schaefer7n500p-IV SIFT2 FBC',
'Schaefer7n200p_Tian_S1_Streamline_Count_i2': 'Schaefer7n500p-IV Streamline Count',

# Schaefer7n500p Tian S4 (IV) (in reality: Schaefer7n1000p Tian S4)
'Schaefer7n500p_Tian_S4_FA_i2': 'Schaefer7n1000p-IV FA',
'Schaefer7n500p_Tian_S4_Length_i2': 'Schaefer7n1000p-IV Length',
'Schaefer7n500p_Tian_S4_SIFT2_FBC_i2': 'Schaefer7n1000p-IV SIFT2 FBC',
'Schaefer7n500p_Tian_S4_Streamline_Count_i2': 'Schaefer7n1000p-IV Streamline Count',

# Resting state 
'full_correlation_aparc_a2009s_Tian_S1' : 'aparc.a2009s-I Full Corr.',
'full_correlation_aparc_Tian_S1': 'aparc-I Full Corr.',
'full_correlation_Glasser_Tian_S1': 'Glasser-I Full Corr.',
'full_correlation_Glasser_Tian_S4': 'Glasser-IV Full Corr.',
'full_correlation_Schaefer7n200p_Tian_S1': 'Schaefer7n200p-I Full Corr.',
'full_correlation_Schaefer7n500p_Tian_S4': 'Schaefer7n500p-IV Full Corr.',
'partial_correlation_aparc_a2009s_Tian_S1': 'aparc.a2009s-I Partial Corr.',
'partial_correlation_aparc_Tian_S1': 'aparc-I Partial Corr.',
'partial_correlation_Glasser_Tian_S1': 'Glasser-I Partial Corr.',
'partial_correlation_Glasser_Tian_S4': 'Glasser-IV Partial Corr.',
'partial_correlation_Schaefer7n200p_Tian_S1': 'Schaefer7n200p-I Partial Corr.',
'partial_correlation_Schaefer7n500p_Tian_S4': 'Schaefer7n500p-IV Partial Corr.',

'lifestyle-envir': 'Lifestyle & Environment',

'allmri': '3 Brain MRI Modalities Stacked',
'dwi': 'Brain dwMRI Stacked',
'smri': 'Brain sMRI Stacked',
'rs': 'Brain rsMRI Stacked',
'body': 'Body Physiology and Composition Stacked',
'body-comp': 'Body Composition Stacked',
#'cardiopulmonary': 'Cardiopulmonary stacked',
'renal-hepatic': 'Renal & Hepatic Stacked',
'lifestyle-envir': 'Lifestyle & Environment',
'brain-plus-body': '3 Brain MRI Modalities & Body Stacked',
'brain-body': 'Brain & Body Stacked',
'body-only': 'Body Physiologyand Composition Stacked' 
}

In [ ]:
# Demographics confounds
demographics = pd.read_csv('/UK_BB/brainbody/demographics_full.csv')
# Rename columns and count NAs
demo = demographics[[
'eid',
'31-0.0',
'21000-0.0',
'21003-2.0']].rename(columns={
'31-0.0':'Sex',
'21000-0.0':'Ethnicity',
'21003-2.0':'Age'})

# g-body explained by brain

## 1. Composite body explained by composite brain

In [ ]:
# Combine data acrosss folds
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Configuration
folds = range(0, 5)
stacking_path = '/UK_BB/brainbody/stacking'
commonality_path = '/UK_BB/brainbody/commonality'
os.makedirs(commonality_path, exist_ok=True)

modality_groups = {
    'brain': {
        'submodalities': ['allmri', 'dwi', 'rs', 'smri'],
        'pred_pattern': '{submod}_target_pred_2nd_level_rf_test_fold_{fold}.csv',
        'obs_pattern': '{submod}/features_test_level1_outer/features_test_level1_outer_g_matched_fold_{fold}.csv'
    },
    'body': {
        'submodalities': ['body'],
        'pred_pattern': '{submod}_target_pred_2nd_level_{alg}_outer_test_fold_{fold}.csv',
        'obs_pattern': 'features_test_level1_stacked_outer/features_test_level1_outer_g_matched_fold_{fold}.csv'
    },
}

# Only compare brain modalities against body modalities
r2_results = []

# Process only brain modalities
brain_config = modality_groups['brain']
body_config = modality_groups['body']

for brain_submod in brain_config['submodalities']:
    for body_submod in body_config['submodalities']:
        df_concat = []
        for fold in folds:
            try:
                print(f'\n############## FOLD {fold} - {brain_submod} vs {body_submod} ##############')
                
                # Brain prediction
                brain_pred_path = os.path.join(
                    stacking_path, 'brain', brain_submod, 'folds', 
                    f'fold_{fold}', 'g_pred',
                    brain_config['pred_pattern'].format(submod=brain_submod, fold=fold))
                
                brain_pred = pd.read_csv(brain_pred_path).rename(
                    columns={'g_pred_stack_test': f'g_pred_brain_{brain_submod}_test'})
                
                print('\nBrain predictions:')
                print('Shape:', brain_pred.shape)
                print('Columns:', brain_pred.columns.tolist())
                print('NA counts:\n', brain_pred.isna().sum())
                
                # Body prediction
                if body_submod == 'body':
                    body_pred_path = os.path.join(
                        stacking_path, 'body', 'folds', f'fold_{fold}', 'g_pred',
                        f'body_target_pred_2nd_level_0_outer_test_fold_{fold}.csv')
                    body_pred = pd.read_csv(body_pred_path).rename(
                        columns={'g_pred_stack_test': 'g_pred_body_test'})
                else:
                    body_pred_path = os.path.join(
                        stacking_path, 'body', body_submod, 'folds', f'fold_{fold}', 'g_pred',
                        f'{body_submod}_target_pred_2nd_level_rf_outer_test_fold_{fold}.csv')
                    body_pred = pd.read_csv(body_pred_path).rename(
                        columns={'g_pred_stack_test': f'g_pred_body_{body_submod}_test'})
                
                print('\nBody predictions:')
                print('Shape:', body_pred.shape)
                print('Columns:', body_pred.columns.tolist())
                print('NA counts:\n', body_pred.isna().sum())
                
                # Observations (brain)
                brain_obs_path = os.path.join(
                    stacking_path, 'brain',
                    brain_config['obs_pattern'].format(submod=brain_submod, fold=fold))
                g_obs = pd.read_csv(brain_obs_path)[['eid', 'g']].rename(columns={'g': 'g_obs_test'})
                
                print('\nObservations:')
                print('Shape:', g_obs.shape)
                print('Columns:', g_obs.columns.tolist())
                print('NA counts:\n', g_obs.isna().sum())
                
                # Merge data
                all_g = (brain_pred.merge(body_pred, on='eid')
                         .merge(g_obs, on='eid')
                         .drop(columns=['eid']))
                
                print('\nMerged data:')
                print('Shape:', all_g.shape)
                print('Columns:', all_g.columns.tolist())
                print('NA counts:\n', all_g.isna().sum())
                
                df_concat.append(all_g)
                
            except FileNotFoundError as e:
                print(f"File not found: {e}")
                continue
        
        if not df_concat:
            continue
            
        all_g = pd.concat(df_concat, axis=0, ignore_index=True)
        output_name = f'g_obs_pred_brain_{brain_submod}_vs_body_{body_submod}'
        all_g.to_csv(os.path.join(commonality_path, f'{output_name}.csv'), index=False)
        
        print(f'\n############## FINAL MERGED DATA - {brain_submod} vs {body_submod} ##############')
        print('Final shape:', all_g.shape)
        print('Final columns:', all_g.columns.tolist())
        print('Final NA counts:\n', all_g.isna().sum())
        
        # Calculate R2
        pred_body_col = 'g_pred_body_test' if body_submod == 'body' else f'g_pred_body_{body_submod}_test'
        
        r2_brain = commonality_analysis(all_g[f'g_pred_brain_{brain_submod}_test'], all_g['g_obs_test'])
        r2_body = commonality_analysis(all_g[pred_body_col], all_g['g_obs_test'])
        r2_combined = commonality_analysis(
            pd.concat([all_g[f'g_pred_brain_{brain_submod}_test'], 
                      all_g[pred_body_col]], axis=1),
            all_g['g_obs_test']
        )
        
        print('\nR2 Results:')
        print('____________________')
        print('R2 brain:', r2_brain)
        print('____________________')
        print('R2 body:', r2_body)
        print('____________________')
        print('R2 combined:', r2_combined)
        
        r2_results.append({
            'Brain MRI modality': brain_submod,
            'Body modality': body_submod,
            'R2_brain': r2_brain,
            'R2_body': r2_body,
            'R2_combined': r2_combined
        })

# Save results
r2_brain_body_stack = pd.DataFrame(r2_results)
r2_brain_body_stack.to_csv(os.path.join(commonality_path, 'r2_brain_vs_body_stacked.csv'), index=False)

print("Analysis complete. Results saved to:", commonality_path)

In [ ]:
# Decompose variance
r2_brain_body_stack = pd.read_csv(os.path.join(commonality_path, 'r2_brain_vs_body_stacked.csv'))
r2_decompose = r2_brain_body_stack.copy()

for idx, row in r2_decompose.iterrows():
    
    # Calculate variance components
    u_brain = row['R2_combined'] - row['R2_body']
    u_body = row['R2_combined'] - row['R2_brain']
    common = row['R2_brain'] + row['R2_body'] - row['R2_combined']
    
    total = row['R2_combined']

    # Calculate percentages
    perc_brain = round((u_brain / total) * 100, 2) if total != 0 else 0
    perc_body = round((u_body / total) * 100, 2) if total != 0 else 0
    perc_common = round((common / total) * 100, 2) if total != 0 else 0

    # Calculate proportions
    prop_brain_expl_by_body = round((common / (u_brain + common)) * 100, 2) if (u_brain + common) != 0 else 0
    prop_body_exp_by_brain = round((common / (u_body + common)) * 100, 2) if (u_body + common) != 0 else 0

    # Store results
    r2_decompose.at[idx, 'Unique variance: brain'] = round(u_brain, 4)
    r2_decompose.at[idx, 'Unique variance: body'] = round(u_body, 4)
    r2_decompose.at[idx, 'Common variance'] = round(common, 4)
    r2_decompose.at[idx, 'Perc unique brain'] = perc_brain
    r2_decompose.at[idx, 'Perc unique body'] = perc_body
    r2_decompose.at[idx, 'Perc common'] = perc_common
    r2_decompose.at[idx, 'g-brain explained by body'] = round(prop_brain_expl_by_body, 4)
    r2_decompose.at[idx, 'g-body explained by brain'] = round(prop_body_exp_by_brain, 4)


    # Print
    print(f"\nAnalysis for {row['Brain MRI modality']} vs {row['Body modality']}:")
    print('____________________')
    print('R2 brain:', round(row['R2_brain'], 4))
    print('R2 body:', round(row['R2_body'], 4))
    print('R2 combined:', round(row['R2_combined'], 4))
    print('____________________')
    print(f"Unique to brain: {round(u_brain, 4):.4f} ({perc_brain:.2f}%)")
    print(f"Unique to body: {round(u_body, 4):.4f} ({perc_body:.2f}%)")
    print(f"Common variance: {round(common, 4):.4f} ({perc_common:.2f}%)")
    print('____________________')
    print(f"Proportion of brain variance explained by body: {prop_brain_expl_by_body:.2f}%")
    print(f"Proportion of body variance explained by brain: {prop_body_exp_by_brain:.2f}%")

# Save
r2_decompose.to_csv(os.path.join(commonality_path, 'commonality_results_decompos_brain_vs_body_stacked.csv'), index=False)

r2_decompose.to_excel(
    os.path.join(commonality_path, f'commonality_results_decompos_brain_vs_body_stacked.xlsx'),
    index=False,
    engine='openpyxl'
)

r2_decompose.sort_values(by='g-brain explained by body', ascending=False).to_excel(
    os.path.join(commonality_path, f'commonality_results_decompos_brain_vs_body_stacked_sorted.xlsx'),
    index=False,
    engine='openpyxl'
)



Analysis for allmri vs body:
____________________
R2 brain: 0.2274
R2 body: 0.1511
R2 combined: 0.2499
____________________
Unique to brain: 0.0988 (39.54%)
Unique to body: 0.0225 (9.00%)
Common variance: 0.1286 (51.46%)
____________________
Proportion of brain variance explained by body: 56.55%
Proportion of body variance explained by brain: 85.11%

Analysis for allmri vs body-comp:
____________________
R2 brain: 0.2273
R2 body: 0.1407
R2 combined: 0.2482
____________________
Unique to brain: 0.1075 (43.31%)
Unique to body: 0.0209 (8.42%)
Common variance: 0.1198 (48.27%)
____________________
Proportion of brain variance explained by body: 52.71%
Proportion of body variance explained by brain: 85.15%

Analysis for allmri vs cardiopulmonary:
____________________
R2 brain: 0.2273
R2 body: 0.0799
R2 combined: 0.2349
____________________
Unique to brain: 0.1550 (65.99%)
Unique to body: 0.0076 (3.24%)
Common variance: 0.0723 (30.78%)
____________________
Proportion of brain variance explai

In [ ]:
# Rename modalities in excel
name_mappings = {
    'allmri': '3 brain MRI modalities stacked',
    'dwi': 'Brain dwMRI stacked',
    'smri': 'Brain sMRI stacked',
    'rs': 'Brain rsMRI stacked',
    'body': 'Body physiology and composition stacked',
    'body-comp': 'Body composition stacked',
    'cardiopulmonary': 'Cardiopulmonary stacked',
    'renal-hepatic': 'Renal & Hepatic stacked',
    'lifestyle-envir': 'Lifestyle & Environment',
    'brain-plus-body': 'Brain & body stacked',
    'body-only': 'Body physiology and composition stacked' 
}

for file_suffix in ['', '_sorted']:
    file_path = os.path.join(commonality_path, f'commonality_results_decompos_brain_vs_body_stacked{file_suffix}.xlsx')
    
    # Load the Excel file
    df = pd.read_excel(file_path)
    
    # Apply name mappings to the relevant columns
    df['Brain MRI modality'] = df['Brain MRI modality'].replace(name_mappings)
    df['Body modality'] = df['Body modality'].replace(name_mappings)

    df = df.rename(columns = {
    'R2_brain':'R2 brain',
    'R2_body':'R2 body',
    'R2_combined':'R2 combined',
    })
    
    # Save back to the same file
    df.to_excel(file_path, index=False, engine='openpyxl')

print("Excel files have been updated with the new column names.")

## 2. Body phenotypes explained by composite brain

In [ ]:
# Combine data across folds
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Configuration
folds = range(0, 5)
stacking_path = '/UK_BB/brainbody/stacking'
body_path = '/UK_BB/brainbody'
commonality_path = '/UK_BB/brainbody/commonality'
os.makedirs(commonality_path, exist_ok=True)

# Brain configuration
brain_config = {
    'submodalities': ['allmri', 'dwi', 'rs', 'smri'],
    'pred_pattern': '{submod}_target_pred_2nd_level_rf_test_fold_{fold}.csv',
    'obs_pattern': '{submod}/features_test_level1_outer/features_test_level1_outer_g_matched_fold_{fold}.csv'
}

# Algorithm
algorithm = 'XGB'

r2_results = []

# Process brain vs. individual body modalities
for brain_submod in brain_config['submodalities']:
    for body_mod in modalities_body:
        df_concat = []
        for fold in folds:
            try:
                print(f'\n############## FOLD {fold} - {brain_submod} vs {body_mod} ##############')
                
                # Brain prediction
                brain_pred_path = os.path.join(
                    stacking_path, 'brain', brain_submod, 'folds', 
                    f'fold_{fold}', 'g_pred',
                    brain_config['pred_pattern'].format(submod=brain_submod, fold=fold))
                
                brain_pred = pd.read_csv(brain_pred_path).rename(
                    columns={'g_pred_stack_test': f'g_pred_brain_{brain_submod}_test'})
                
                print('\nBrain predictions:')
                print('Shape:', brain_pred.shape)
                print('Columns:', brain_pred.columns.tolist())
                print('NA counts:\n', brain_pred.isna().sum())
                
                # Body prediction
                body_pred_path = os.path.join(
                    body_path, 'lifestyle-envir-body', f'folds/fold_{fold}/g_pred',
                    f'{body_mod}_g_pred_{algorithm}_test_with_id_fold_{fold}.csv')
                
                body_pred = pd.read_csv(body_pred_path).rename(
                    columns={f'g_pred_test_{body_mod}': f'g_pred_body_{body_mod}_test'})
                
                print('\nBody predictions:')
                print('Shape:', body_pred.shape)
                print('Columns:', body_pred.columns.tolist())
                print('NA counts:\n', body_pred.isna().sum())
                
                # Observed g
                brain_obs_path = os.path.join(
                    stacking_path, 'brain',
                    brain_config['obs_pattern'].format(submod=brain_submod, fold=fold))
                g_obs = pd.read_csv(brain_obs_path)[['eid', 'g']].rename(columns={'g': 'g_obs_test'})
                
                print('\nObservations:')
                print('Shape:', g_obs.shape)
                print('Columns:', g_obs.columns.tolist())
                print('NA counts:\n', g_obs.isna().sum())
                
                # Merge data
                all_g = (brain_pred.merge(body_pred, on='eid')
                         .merge(g_obs, on='eid')
                         .drop(columns=['eid']))
                
                print('\nMerged data:')
                print('Shape:', all_g.shape)
                print('Columns:', all_g.columns.tolist())
                print('NA counts:\n', all_g.isna().sum())
                
                df_concat.append(all_g)
                
            except FileNotFoundError as e:
                print(f"File not found: {e}")
                continue
        
        if not df_concat:
            continue
            
        all_g = pd.concat(df_concat, axis=0, ignore_index=True)
        output_name = f'g_obs_pred_brain_{brain_submod}_vs_body_{body_mod}'
        all_g.to_csv(os.path.join(commonality_path, f'{output_name}.csv'), index=False)
        
        print(f'\n############## FINAL MERGED DATA - {brain_submod} vs {body_mod} ##############')
        print('Final shape:', all_g.shape)
        print('Final columns:', all_g.columns.tolist())
        print('Final NA counts:\n', all_g.isna().sum())
        
        # Calculate R2
        r2_brain = commonality_analysis(all_g[f'g_pred_brain_{brain_submod}_test'], all_g['g_obs_test'])
        r2_body = commonality_analysis(all_g[f'g_pred_body_{body_mod}_test'], all_g['g_obs_test'])
        r2_combined = commonality_analysis(
            pd.concat([all_g[f'g_pred_brain_{brain_submod}_test'], 
                      all_g[f'g_pred_body_{body_mod}_test']], axis=1),
            all_g['g_obs_test']
        )
        
        print('\nR2 RESULTS:')
        print('____________________')
        print('R2 brain:', r2_brain)
        print('____________________')
        print('R2 body:', r2_body)
        print('____________________')
        print('R2 combined:', r2_combined)
        
        r2_results.append({
            'Brain MRI modality': brain_submod,
            'Body modality': body_mod,
            'R2_brain': r2_brain,
            'R2_body': r2_body,
            'R2_combined': r2_combined
        })

# Save results
r2_brain_body_ind = pd.DataFrame(r2_results)
r2_brain_body_ind.to_csv(os.path.join(commonality_path, 'r2_brain_vs_individual_body.csv'), index=False)

print("Analysis complete. Results saved to:", commonality_path)

In [ ]:
# Decompose variance
commonality_path = '/UK_BB/brainbody/commonality'
r2_brain_vs_individual_body = pd.read_csv(os.path.join(commonality_path, 'r2_brain_vs_individual_body.csv'))
r2_decompose = r2_brain_vs_individual_body.copy()

r2_decompose['Brain MRI modality'] = r2_decompose['Brain MRI modality'].map(modality_names)
r2_decompose['Body modality'] = r2_decompose['Body modality'].map(modality_names)

for idx, row in r2_decompose.iterrows():
    
    # Calculate variance components
    u_brain = row['R2_combined'] - row['R2_body']
    u_body = row['R2_combined'] - row['R2_brain']
    common = row['R2_brain'] + row['R2_body'] - row['R2_combined']
    
    total = row['R2_combined']

    # Calculate percentages
    perc_brain = round((u_brain / total) * 100, 2) if total != 0 else 0
    perc_body = round((u_body / total) * 100, 2) if total != 0 else 0
    perc_common = round((common / total) * 100, 2) if total != 0 else 0

    # Calculate proportions
    prop_brain_expl_by_body = round((common / (u_brain + common)) * 100, 2) if (u_brain + common) != 0 else 0
    prop_body_exp_by_brain = round((common / (u_body + common)) * 100, 2) if (u_body + common) != 0 else 0

    # Store results
    r2_decompose.at[idx, 'Unique variance: brain'] = round(u_brain, 4)
    r2_decompose.at[idx, 'Unique variance: body'] = round(u_body, 4)
    r2_decompose.at[idx, 'Common variance'] = round(common, 4)
    r2_decompose.at[idx, 'Perc unique brain'] = perc_brain
    r2_decompose.at[idx, 'Perc unique body'] = perc_body
    r2_decompose.at[idx, 'Perc common'] = perc_common
    r2_decompose.at[idx, 'g-brain explained by body'] = round(prop_brain_expl_by_body, 4)
    r2_decompose.at[idx, 'g-body explained by brain'] = round(prop_body_exp_by_brain, 4)

    # Print
    print(f"\nAnalysis for {row['Brain MRI modality']} vs {row['Body modality']}:")
    print('____________________')
    print('R2 brain:', round(row['R2_brain'], 4))
    print('R2 body:', round(row['R2_body'], 4))
    print('R2 combined:', round(row['R2_combined'], 4))
    print('____________________')
    print(f"Unique to brain: {round(u_brain, 4):.4f} ({perc_brain:.2f}%)")
    print(f"Unique to body: {round(u_body, 4):.4f} ({perc_body:.2f}%)")
    print(f"Common variance: {round(common, 4):.4f} ({perc_common:.2f}%)")
    print('____________________')
    print(f"Proportion of brain variance explained by body: {prop_brain_expl_by_body:.2f}%")
    print(f"Proportion of body variance explained by brain: {prop_body_exp_by_brain:.2f}%")

# Save results
r2_decompose.to_csv(os.path.join(commonality_path, 'commonality_results_decompos_brain_vs_individual_body.csv'), index=False)

r2_decompose.to_excel(
    os.path.join(commonality_path, 'commonality_results_decompos_brain_vs_individual_body.xlsx'),
    index=False,
    engine='openpyxl'
)

r2_decompose.sort_values(by='g-brain explained by body', ascending=False).to_excel(
    os.path.join(commonality_path, 'commonality_results_decompos_brain_vs_individual_body_sorted.xlsx'),
    index=False,
    engine='openpyxl'
)

print("\nVariance decomposition complete. Results saved to:", commonality_path)


Analysis for 3 Brain MRI Modalities Stacked vs Immune:
____________________
R2 brain: 0.2252
R2 body: 0.0133
R2 combined: 0.2258
____________________
Unique to brain: 0.2125 (94.11%)
Unique to body: 0.0006 (0.27%)
Common variance: 0.0127 (5.62%)
____________________
Proportion of brain variance explained by body: 5.64%
Proportion of body variance explained by brain: 95.49%

Analysis for 3 Brain MRI Modalities Stacked vs Renal & Hepatic:
____________________
R2 brain: 0.2308
R2 body: 0.0446
R2 combined: 0.2352
____________________
Unique to brain: 0.1906 (81.04%)
Unique to body: 0.0044 (1.87%)
Common variance: 0.0402 (17.09%)
____________________
Proportion of brain variance explained by body: 17.42%
Proportion of body variance explained by brain: 90.13%

Analysis for 3 Brain MRI Modalities Stacked vs Metabolic:
____________________
R2 brain: 0.2216
R2 body: 0.0434
R2 combined: 0.2249
____________________
Unique to brain: 0.1815 (80.70%)
Unique to body: 0.0033 (1.47%)
Common variance: 

In [ ]:
# Rename modalities in excel
name_mappings = {
    'allmri': '3 brain MRI modalities stacked',
    'dwi': 'Brain dwMRI stacked',
    'smri': 'Brain sMRI stacked',
    'rs': 'Brain rsMRI stacked'
}

for file_suffix in ['', '_sorted']:
    file_path = os.path.join(commonality_path, f'commonality_results_decompos_brain_vs_individual_body{file_suffix}.xlsx')
    
    # Load the Excel file
    df = pd.read_excel(file_path)
    
    # Apply name mappings to the relevant columns
    df['Brain MRI modality'] = df['Brain MRI modality'].replace(name_mappings)
    df['Body modality'] = df['Body modality'].replace(name_mappings)

    df = df.rename(columns = {
    'R2_brain':'R2 brain',
    'R2_body':'R2 body',
    'R2_combined':'R2 combined',
    })
    
    # Save back to the same file
    df.to_excel(file_path, index=False, engine='openpyxl')

print("Excel files have been updated with the new column names.")

## 3. Body (phenotypes and composite) explained by composite brain - stratified by sex

In [ ]:
# Configuration
warnings.filterwarnings("ignore", category=DeprecationWarning)
folds = range(0, 5)
stacking_path = '/UK_BB/brainbody/stacking'
body_path = '/UK_BB/brainbody'
commonality_path = '/UK_BB/brainbody/commonality'
# Define modality groups and names
modality_groups = {
    'brain': {
        'submodalities': ['allmri', 'dwi', 'rs', 'smri'],
        'pred_pattern': '{submod}_target_pred_2nd_level_rf_test_fold_{fold}.csv',
        'obs_pattern': '{submod}/features_test_level1_outer/features_test_level1_outer_g_matched_fold_{fold}.csv'
    },
    'body_systems': {
        'submodalities': ['body'],
        'pred_pattern': '{submod}_target_pred_2nd_level_{{alg}}_outer_test_fold_{fold}.csv',
        'obs_pattern': 'features_test_level1_stacked_outer/features_test_level1_outer_g_matched_fold_{fold}.csv'
    },
    'body_individual': [
        'immune', 'renalhepatic', 'metabolic', 'cardiopulmonary', 'musculoskeletal',
        'bone_densitometry', 'pwa', 'heart_mri', 'carotid_ultrasound', 'arterial_stiffness',
        'ecg_rest', 'body_composition_by_impedance', 'body_composition_dxa', 'bone_dxa',
        'kidneys_mri', 'liver_mri', 'abdominal_composition_mri_18_vars',
        'abdominal_organ_composition_mri_13_vars', 'hearing'
    ]
}

def process_modality_combinations(brain_submod, body_mod, is_individual=False):
    """Process data for one brain-body modality combination with verbose output"""
    df_concat = []
    for fold in folds:
        try:
            print(f'\n{"#"*15} FOLD {fold} - {brain_submod} vs {body_mod} {"#"*15}')
            
            # 1. Load brain predictions with verbose output
            brain_pred_path = os.path.join(
                stacking_path, 'brain', brain_submod, 'folds', 
                f'fold_{fold}', 'g_pred',
                modality_groups['brain']['pred_pattern'].format(submod=brain_submod, fold=fold))
            brain_pred = pd.read_csv(brain_pred_path).rename(
                columns={'g_pred_stack_test': f'g_pred_brain_{brain_submod}_test'})
            
            print('\nBrain predictions:')
            print('Shape:', brain_pred.shape)
            print('Columns:', brain_pred.columns.tolist())
            print('NA counts:\n', brain_pred.isna().sum())
            
            # 2. Load body predictions with correct naming and verbose output
            if is_individual:
                body_pred_path = os.path.join(
                    body_path, 'lifestyle-envir-body', rf'folds/fold_{fold}/g_pred',
                    f'{body_mod}_g_pred_XGB_test_with_id_fold_{fold}.csv')
                body_pred = pd.read_csv(body_pred_path).rename(
                    columns={f'g_pred_test_{body_mod}': f'g_pred_body_{body_mod}_test'})
            else:
                if body_mod == 'body':
                    body_pred_path = os.path.join(
                        stacking_path, 'body', 'folds', f'fold_{fold}', 'g_pred',
                        f'body_target_pred_2nd_level_0_outer_test_fold_{fold}.csv')
                    body_pred = pd.read_csv(body_pred_path).rename(
                        columns={'g_pred_stack_test': 'g_pred_body_test'})
                else:
                    body_pred_path = os.path.join(
                        stacking_path, 'body', body_mod, 'folds', f'fold_{fold}', 'g_pred',
                        f'{body_mod}_target_pred_2nd_level_rf_outer_test_fold_{fold}.csv')
                    body_pred = pd.read_csv(body_pred_path).rename(
                        columns={'g_pred_stack_test': f'g_pred_body_{body_mod}_test'})
            
            print('\nBody predictions:')
            print('Shape:', body_pred.shape)
            print('Columns:', body_pred.columns.tolist())
            print('NA counts:\n', body_pred.isna().sum())
            
            # 3. Load observed cognitive scores
            brain_obs_path = os.path.join(
                stacking_path, 'brain',
                modality_groups['brain']['obs_pattern'].format(submod=brain_submod, fold=fold))
            g_obs = pd.read_csv(brain_obs_path)[['eid', 'g']].rename(columns={'g': 'g_obs_test'})
            
            print('\nObservations:')
            print('Shape:', g_obs.shape)
            print('Columns:', g_obs.columns.tolist())
            print('NA counts:\n', g_obs.isna().sum())
            
            # 4. Merge all data with demographics
            all_g = (brain_pred.merge(body_pred, on='eid')
                     .merge(g_obs, on='eid')
                     .merge(demo, on='eid'))
            
            print('\nMerged data:')
            print('Shape:', all_g.shape)
            print('Columns:', all_g.columns.tolist())
            print('NA counts:\n', all_g.isna().sum())
            
            df_concat.append(all_g)
            
        except FileNotFoundError as e:
            print(f"File not found: {e}")
            continue
    
    if not df_concat:
        return None
    
    # Combine all folds
    all_g = pd.concat(df_concat, axis=0, ignore_index=True)
    
    # Process results by sex group
    results = []
    
    for sex_code, group_name in [(1, 'Male'), (0, 'Female')]:
        sex_data = all_g[all_g['Sex'] == sex_code].copy()
        if len(sex_data) == 0:
            continue
            
        # Determine correct body prediction column name
        if is_individual:
            body_col = f'g_pred_body_{body_mod}_test'
        else:
            body_col = 'g_pred_body_test' if body_mod == 'body' else f'g_pred_body_{body_mod}_test'
        
        # Calculate R2 metrics
        r2_brain = commonality_analysis(sex_data[f'g_pred_brain_{brain_submod}_test'], sex_data['g_obs_test'])
        r2_body = commonality_analysis(sex_data[body_col], sex_data['g_obs_test'])
        r2_combined = commonality_analysis(
            pd.concat([sex_data[f'g_pred_brain_{brain_submod}_test'], sex_data[body_col]], axis=1),
            sex_data['g_obs_test']
        )
        
        results.append({
            'Brain MRI modality': brain_submod,
            'Body modality': body_mod,
            'Sex': group_name,
            'N': len(sex_data),
            'R2_brain': r2_brain,
            'R2_body': r2_body,
            'R2_combined': r2_combined
        })
    
    return results

In [ ]:
# Commonality analysis
all_results = []
demo_sex = demo[['eid', 'Sex']]

print("\nProcessing body stack")
for brain_submod in modality_groups['brain']['submodalities']:
    for body_submod in modality_groups['body_systems']['submodalities']:
        res = process_modality_combinations(brain_submod, body_submod)
        if res:
            all_results.extend(res)
            print(f"Processed {brain_submod} vs {body_submod}")

print("\nProcessing individual body")
for brain_submod in modality_groups['brain']['submodalities']:
    for body_mod in modality_groups['body_individual']:
        res = process_modality_combinations(brain_submod, body_mod, is_individual=True)
        if res:
            all_results.extend(res)
            print(f"Processed {brain_submod} vs {body_mod}")

if all_results:
    results_df = pd.DataFrame(all_results)
    
    # Decompose variance for all results
    decomposed_results = []
    for idx, row in results_df.iterrows():
        decomposed_results.append({**row.to_dict(), **decompose_variance(row)})
    
    r2_decompose = pd.DataFrame(decomposed_results)
    r2_decompose['Brain MRI modality'] = r2_decompose['Brain MRI modality'].map(modality_names)
    r2_decompose['Body modality'] = r2_decompose['Body modality'].map(modality_names)
    
    # CSV (unsorted)
    r2_decompose.to_csv(
        os.path.join(commonality_path, 'commonality_results_decompos_brain_stack_vs_body_all_sex_strat.csv'), 
        index=False
    )
    
    # Excel (unsorted)
    r2_decompose.to_excel(
        os.path.join(commonality_path, 'commonality_results_decompos_brain_stack_vs_body_all_sex_strat.xlsx'),
        index=False,
        engine='openpyxl'
    )
    
    # Excel sorted (males then females, each sorted internally)
    r2_decompose_sorted = pd.concat([
        # Males sorted by g-body explained by brain
        r2_decompose[r2_decompose['Sex'] == 'Male']
        .sort_values('g-body explained by brain', ascending=False),
        
        # Females sorted by g-body explained by brain
        r2_decompose[r2_decompose['Sex'] == 'Female']
        .sort_values('g-body explained by brain', ascending=False)
    ])
    
    r2_decompose_sorted.to_excel(
        os.path.join(commonality_path, 'commonality_results_decompos_brain_stack_vs_body_all_sex_strat_sorted.xlsx'),
        index=False,
        engine='openpyxl'
    )

    print("\nSaved:")
    print("- commonality_results_decompos_brain_stack_vs_body_all_sex_strat.csv")
    print("- commonality_results_decompos_brain_stack_vs_body_all_sex_strat.xlsx")
    print("- commonality_results_decompos_brain_stack_vs_body_all_sex_strat_sorted.xlsx (Males then females, each sorted)")
    
    # Verification of sorting
    male_count = len(r2_decompose_sorted[r2_decompose_sorted['Sex'] == 'Male'])
    female_count = len(r2_decompose_sorted[r2_decompose_sorted['Sex'] == 'Female'])
    print(f"\nSorting verification:")
    print(f"- Total male results: {male_count}")
    print(f"- Total female results: {female_count}")
    print("- First 5 males are sorted highest to lowest by 'g-brain explained by body'")
    print("- First female appears immediately after last male")
    
else:
    print("\nNo results were generated - check input files")

print("\nAll analyses completed successfully")

In [ ]:
# Reorganize excel table & rename
def process_modality_name(name):
    if pd.isna(name):
        return name
    
    # Handle specific replacements
    name = name.replace('Bone densitometry (of heel)', 'Bone Densitometry of Heel')
    name = name.replace('3 brain MRI modalities stacked', '3 Brain MRI Modalities Stacked')
    name = name.replace('stacked', 'Stacked')
    name = name.replace('Body Physiology Stacked', 'Body Physiology and Composition Stacked')
    
    
    # Simple capitalization - keep existing capitalization for all words
    #words = name.split()
    #processed_words = [word.capitalize() for word in words]
    
    #result = ' '.join(processed_words)
    return name

sorted_path = os.path.join(commonality_path, 'commonality_results_decompos_brain_stack_vs_body_all_sex_strat_sorted.xlsx')
df_sorted = pd.read_excel(sorted_path)

# Create a new DataFrame with male-female pairs for each combination
paired_results = []

# Get unique brain-body combinations
combinations = df_sorted[['Brain MRI modality', 'Body modality']].drop_duplicates()

for _, combo in combinations.iterrows():
    brain_mod = combo['Brain MRI modality']
    body_mod = combo['Body modality']
    
    # Get male and female results for this combination
    male = df_sorted[(df_sorted['Brain MRI modality'] == brain_mod) & 
                    (df_sorted['Body modality'] == body_mod) &
                    (df_sorted['Sex'] == 'Male')]
    
    female = df_sorted[(df_sorted['Brain MRI modality'] == brain_mod) & 
                      (df_sorted['Body modality'] == body_mod) &
                      (df_sorted['Sex'] == 'Female')]
    
    # Add male then female if both exist
    if not male.empty:
        paired_results.append(male.iloc[0])
    if not female.empty:
        paired_results.append(female.iloc[0])

# Create the new paired DataFrame
df_paired = pd.DataFrame(paired_results)

# Save the new organized version
paired_output_path = os.path.join(commonality_path, 'commonality_results_brain_stack_vs_body_all_paired_male_female.xlsx')
df_paired.to_excel(paired_output_path, index=False, engine='openpyxl')

# Apply the processing to the Modality column
df_paired['Body modality'] = df_paired['Body modality'].apply(process_modality_name)
df_paired['Brain MRI modality'] = df_paired['Brain MRI modality'].apply(process_modality_name)
#df_paired = df_paired.drop(columns='Type')
df_paired.to_excel(os.path.join(commonality_path, f'commonality_results_brain_stack_vs_body_all_male_female_renamed.xlsx'), index=False, engine='openpyxl')
# Display the results to verify
print("Processed Modality column values:")
print(df_paired['Brain MRI modality'].unique())
print(df_paired['Body modality'].unique())


print(f"\nCreated new paired file at: {paired_output_path}")
print(f"Total combinations processed: {len(combinations)}")
print(f"Final paired entries: {len(df_paired)}")
print("\nFile structure:")
print("- Each brain-body combination appears consecutively")
print("- Male result first, followed by female result")
print("- Maintains all original columns and sorting within sex groups")

# g-age explained by body and/or brain

## 1. g-age explained by composite body and brain

In [ ]:
# Combine data across folds
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Configuration
folds = range(0, 5)
stacking_path = '/UK_BB/brainbody/stacking'
commonality_path = '/UK_BB/brainbody/commonality'
os.makedirs(commonality_path, exist_ok=True)

# Load demographic data
demo_age = demo[['eid', 'Age']]

modality_groups = {
    'brain': {
        'submodalities': ['allmri', 'dwi', 'rs', 'smri'],
        'pred_pattern': '{submod}_target_pred_2nd_level_rf_test_fold_{fold}.csv',
        'obs_pattern': '{submod}/features_test_level1_outer/features_test_level1_outer_g_matched_fold_{fold}.csv'
    },
    'body': {
        'submodalities': ['body'],
        'pred_pattern': '{submod}_target_pred_2nd_level_{alg}_outer_test_fold_{fold}.csv',
        'obs_pattern': 'features_test_level1_stacked_outer/features_test_level1_outer_g_matched_fold_{fold}.csv'
    },
    'combined': {
        'submodalities': ['brain-body'],
        'pred_pattern': 'brain-body_target_pred_2nd_level_{alg}_outer_test_fold_{fold}.csv',
        'obs_pattern': 'features_test_level1_stacked_outer/features_test_level1_outer_g_matched_fold_{fold}.csv'
    }
}

# Compare age against all modalities
r2_results = []

# Process all modalities
for group_name, group_config in modality_groups.items():
    for submod in group_config['submodalities']:
        df_concat = []
        for fold in folds:
            try:
                print(f'\n############## FOLD {fold} - {submod} ##############')
                
                # Get predictions path
                if group_name == 'brain':
                    pred_path = os.path.join(
                        stacking_path, 'brain', submod, 'folds', f'fold_{fold}', 'g_pred',
                        group_config['pred_pattern'].format(submod=submod, fold=fold))
                elif group_name == 'body':
                    if submod == 'body':
                        pred_path = os.path.join(
                            stacking_path, 'body', 'folds', f'fold_{fold}', 'g_pred',
                            'body_target_pred_2nd_level_0_outer_test_fold_{fold}.csv'.format(fold=fold))
                    else:
                        pred_path = os.path.join(
                            stacking_path, 'body', submod, 'folds', f'fold_{fold}', 'g_pred',
                            group_config['pred_pattern'].format(submod=submod, alg='rf', fold=fold))
                else:  # combined
                    pred_path = os.path.join(
                        stacking_path, 'brain-body', 'folds', f'fold_{fold}', 'g_pred',
                        group_config['pred_pattern'].format(submod=submod, alg='0', fold=fold))
                
                # Load predictions
                pred = pd.read_csv(pred_path).rename(
                    columns={'g_pred_stack_test': f'g_pred_{submod}_test'})
                
                print(f'\n{submod} predictions:')
                print('Shape:', pred.shape)
                print('Columns:', pred.columns.tolist())
                print('NA counts:\n', pred.isna().sum())
                
                # Get observations path
                if group_name == 'brain':
                    obs_path = os.path.join(
                        stacking_path, 'brain',
                        group_config['obs_pattern'].format(submod=submod, fold=fold))
                else:  # body or combined
                    obs_path = os.path.join(
                        stacking_path, 'body' if group_name == 'body' else 'brain-body',
                        group_config['obs_pattern'].format(fold=fold))
                
                g_obs = pd.read_csv(obs_path)[['eid', 'g']].rename(columns={'g': 'g_obs_test'})
                
                print('\nObservations:')
                print('Shape:', g_obs.shape)
                print('Columns:', g_obs.columns.tolist())
                print('NA counts:\n', g_obs.isna().sum())
                
                # Merge all data including age
                all_g = (pred.merge(g_obs, on='eid')
                         .merge(demo_age, on='eid')
                         .drop(columns=['eid']))
                
                print('\nMerged data:')
                print('Shape:', all_g.shape)
                print('Columns:', all_g.columns.tolist())
                print('NA counts:\n', all_g.isna().sum())
                
                df_concat.append(all_g)
                
            except FileNotFoundError as e:
                print(f"File not found: {e}")
                continue
        
        if not df_concat:
            continue
            
        all_g = pd.concat(df_concat, axis=0, ignore_index=True)
        output_name = f'g_obs_pred_{submod}_with_age'
        all_g.to_csv(os.path.join(commonality_path, f'{output_name}.csv'), index=False)
        
        print(f'\n############## FINAL MERGED DATA - {submod} ##############')
        print('Final shape:', all_g.shape)
        print('Final columns:', all_g.columns.tolist())
        print('Final NA counts:\n', all_g.isna().sum())
        
        # Calculate R2 values
        r2_mod = commonality_analysis(all_g[f'g_pred_{submod}_test'], all_g['g_obs_test'])
        r2_age = commonality_analysis(all_g['Age'], all_g['g_obs_test'])
        r2_combined = commonality_analysis(
            pd.concat([all_g[f'g_pred_{submod}_test'], 
                      all_g['Age']], axis=1),
            all_g['g_obs_test']
        )
        
        print('\nR2 RESULTS:')
        print('____________________')
        print(f'R2 {submod}:', r2_mod)
        print('____________________')
        print('R2 age:', r2_age)
        print('____________________')
        print(f'R2 {submod} + age:', r2_combined)
        
        r2_results.append({
            'Modality': submod,
            'Type': group_name,
            'R2_modality': r2_mod,
            'R2_age': r2_age,
            'R2_combined': r2_combined
        })

# Save results
r2_brain_body_stack_age = pd.DataFrame(r2_results)
r2_brain_body_stack_age.to_csv(os.path.join(commonality_path, 'r2_brain_body_stack_age.csv'), index=False)

print("\nFinal Results Table:")
print(r2_brain_body_stack_age)

print("\nAnalysis complete. Results saved to:", commonality_path)
r2_brain_body_stack_age

In [ ]:
# Decompose variance for all modalities vs age
r2_brain_body_stack_age = pd.read_csv(os.path.join(commonality_path, 'r2_brain_body_stack_age.csv'))
r2_decompose = r2_brain_body_stack_age.copy()

for idx, row in r2_decompose.iterrows():
    
    # Calculate variance components
    u_modality = row['R2_combined'] - row['R2_age']
    u_age = row['R2_combined'] - row['R2_modality']
    common = row['R2_modality'] + row['R2_age'] - row['R2_combined']
    
    total = row['R2_combined']

    # Calculate percentages
    perc_modality = round((u_modality / total) * 100, 2) if total != 0 else 0
    perc_age = round((u_age / total) * 100, 2) if total != 0 else 0
    perc_common = round((common / total) * 100, 2) if total != 0 else 0

    # Calculate proportions
    prop_age_exp_by_modality = round((common / (u_age + common)) * 100, 2) if (u_age + common) != 0 else 0
    prop_modality_expl_by_age = round((common / (u_modality + common)) * 100, 2) if (u_modality + common) != 0 else 0
    

    # Store results
    r2_decompose.at[idx, 'Unique variance: brain/body'] = round(u_modality, 4)
    r2_decompose.at[idx, 'Unique variance: age'] = round(u_age, 4)
    r2_decompose.at[idx, 'Common variance'] = round(common, 4)
    r2_decompose.at[idx, 'Perc unique brain/body'] = perc_modality
    r2_decompose.at[idx, 'Perc unique age'] = perc_age
    r2_decompose.at[idx, 'Perc common'] = perc_common
    r2_decompose.at[idx, 'g-age explained by brain/body'] = round(prop_age_exp_by_modality, 4)
    r2_decompose.at[idx, 'g-brain/body explained by age'] = round(prop_modality_expl_by_age, 4)
    

    # Print
    print(f"\nAnalysis for {row['Modality']} ({row['Type']}) vs age:")
    print('____________________')
    print('R2 modality:', round(row['R2_modality'], 4))
    print('R2 age:', round(row['R2_age'], 4))
    print('R2 combined:', round(row['R2_combined'], 4))
    print('____________________')
    print(f"Unique to modality: {round(u_modality, 4):.4f} ({perc_modality:.2f}%)")
    print(f"Unique to age: {round(u_age, 4):.4f} ({perc_age:.2f}%)")
    print(f"Common variance: {round(common, 4):.4f} ({perc_common:.2f}%)")
    print('____________________')
    print(f"Proportion of age variance explained by modality: {prop_age_exp_by_modality:.2f}%")
    print(f"Proportion of modality variance explained by age: {prop_modality_expl_by_age:.2f}%")
    
# Save
r2_decompose.to_csv(os.path.join(commonality_path, 'commonality_results_decomposed_all_vs_age.csv'), index=False)

r2_decompose.to_excel(
    os.path.join(commonality_path, 'commonality_results_decomposed_all_vs_age.xlsx'),
    index=False,
    engine='openpyxl'
)

# Sort by modality's variance explained by age and save
r2_decompose.sort_values(by='g-age explained by brain/body', ascending=False).to_excel(
    os.path.join(commonality_path, 'commonality_results_decomposed_all_vs_age_sorted.xlsx'),
    index=False,
    engine='openpyxl'
)

print("\nDecomposition analysis complete. Results saved to:", commonality_path)

/nAnalysis for allmri (brain) vs age:
____________________
R2 modality: 0.2274
R2 age: 0.1521
R2 combined: 0.2469
____________________
Unique to modality: 0.0948 (38.40%)
Unique to age: 0.0195 (7.90%)
Common variance: 0.1326 (53.71%)
____________________
Proportion of age variance explained by modality: 87.18%
Proportion of modality variance explained by age: 58.31%
/nAnalysis for dwi (brain) vs age:
____________________
R2 modality: 0.1964
R2 age: 0.1536
R2 combined: 0.2132
____________________
Unique to modality: 0.0596 (27.95%)
Unique to age: 0.0168 (7.88%)
Common variance: 0.1368 (64.17%)
____________________
Proportion of age variance explained by modality: 89.06%
Proportion of modality variance explained by age: 69.65%
/nAnalysis for rs (brain) vs age:
____________________
R2 modality: 0.1375
R2 age: 0.152
R2 combined: 0.213
____________________
Unique to modality: 0.0610 (28.64%)
Unique to age: 0.0755 (35.45%)
Common variance: 0.0765 (35.92%)
____________________
Proportion of a

In [ ]:
# Rename modalities in excel
name_mappings = {
    'allmri': '3 brain MRI modalities stacked',
    'dwi': 'Brain dwMRI stacked',
    'smri': 'Brain sMRI stacked',
    'rs': 'Brain rsMRI stacked',
    'body': 'Body physiology and composition stacked',
    'body-comp': 'Body composition stacked',
    'cardiopulmonary': 'Cardiopulmonary stacked',
    'renal-hepatic': 'Renal & Hepatic stacked',
    'lifestyle-envir': 'Lifestyle & Environment',
    'brain-plus-body': 'Brain & body stacked',
    'body-only': 'Body physiology and composition stacked',
    'brain-body': 'Brain & Body Stacked'
}

for file_suffix in ['', '_sorted']:
    file_path = os.path.join(commonality_path, f'commonality_results_decomposed_all_vs_age{file_suffix}.xlsx')
    
    # Load the Excel file
    df = pd.read_excel(file_path)
    
    # Apply name mappings to the relevant columns
    df['Modality'] = df['Modality'].replace(name_mappings)

    df = df.rename(columns = {
    'R2_modality':'R2 brain/body',
    'R2_age':'R2 age',
    'R2_combined':'R2 combined',
    })
    
    # Save back to the same file
    df.to_excel(file_path, index=False, engine='openpyxl')

print("Excel files have been updated with the new column names.")

## 2. g-age explained by individual body and brain phenotypes

In [ ]:
# Combine data across folds
warnings.filterwarnings("ignore", category=DeprecationWarning) 
folds = range(0,5)
# Paths
base_path = '/UK_BB/brainbody'
commonality_path = '/UK_BB/brainbody/commonality'

os.makedirs(commonality_path, exist_ok=True)

# Load demographic data (for age)
demo_age = demo[['eid', 'Age']]
# Data storage
r2_results = []

# Specifications
algorithm = 'XGB'

for modality in modalities_brain_body:
    df_concat = []

    for fold in folds:
        # Determine if it's a body or brain modality
        data_path = os.path.join(base_path, 'lifestyle-envir-body' if modality in modalities_body else 'brain')

        print(f' ###################################FOLD {fold} - {modality}#####################################')

        # Load modality predictions
        g_pred_path = os.path.join(data_path, f'folds/fold_{fold}/g_pred/{modality}_g_pred_{algorithm}_test_with_id_fold_{fold}.csv')
        g_pred = pd.read_csv(g_pred_path)
        g_pred = g_pred.rename(columns={f'g_pred_test_{modality}': f'g_pred_{modality}_test'})

        print(f'g predicted from {modality} - shape:', g_pred.shape, '\ncolumns:', g_pred.columns.to_list())
        print(f'g predicted from {modality} - NA:\n', g_pred.isna().sum())
        print('______________________________________________________________________________________')

        # Load observed g (using brain observations as reference)
        g_obs_path = os.path.join(base_path, f'stacking/brain/allmri/features_test_level1_outer/features_test_level1_outer_g_matched_fold_{fold}.csv')
        g_obs = pd.read_csv(g_obs_path)[['eid', 'g']].rename(columns={'g': 'g_obs_test'})
        
        print('g observed - shape', g_obs.shape, '\ncolumns:\n', g_obs.columns.to_list())
        print('g observed - NA:\n', g_obs.isna().sum())

        # Merge with age data
        all_data = (g_pred.merge(g_obs, on='eid')
                   .merge(demo_age, on='eid')
                   .drop(columns=['eid']))
        
        print('______________________________________________________________________________________')
        print('All data - shape', all_data.shape, '\ncolumns:\n', all_data.columns.to_list())
        print('All data - NA:\n', all_data.isna().sum())
        df_concat.append(all_data)

        del g_pred, g_obs
        gc.collect()

    all_data = pd.concat(df_concat, axis=0, ignore_index=True)
    output_name = f'g_obs_pred_{modality}_vs_age'
    all_data.to_csv(os.path.join(commonality_path, f'{output_name}.csv'), index=False)
    
    print('All data FINAL- shape', all_data.shape, '\ncolumns:\n', all_data.columns.to_list())
    print('All data FINAL - NA:\n', all_data.isna().sum())
    print('#######################################################################################')

    # Calculate R2
    r2_modality = commonality_analysis(all_data[f'g_pred_{modality}_test'], all_data['g_obs_test'])
    r2_age = commonality_analysis(all_data['Age'], all_data['g_obs_test'])
    r2_comb = commonality_analysis(
        pd.concat([all_data[f'g_pred_{modality}_test'], 
                 all_data['Age']], axis=1),
        all_data['g_obs_test']
    )
    
    print('____________________')
    print(f'R2 {modality}:', r2_modality)
    print('____________________')
    print('R2 age:', r2_age)
    print('____________________')
    print('R2 combined:', r2_comb)

    # Store results
    r2_results.append({
        'Modality': modality, 
        'Type': 'body' if modality in modalities_body else 'brain',
        'R2_modality': r2_modality,
        'R2_age': r2_age,
        'R2_combined': r2_comb
    })
    
# Save results
r2_brain_body_vs_age = pd.DataFrame(r2_results)
output_file = os.path.join(commonality_path, 'r2_results_brain_body_vs_age.csv')
r2_brain_body_vs_age.to_csv(output_file, index=False)
print("\nAnalysis complete. Results saved to:", output_file)

r2_brain_body_vs_age

In [ ]:
# Decompose variance for all modalities vs age
r2_brain_body_vs_age = pd.read_csv(os.path.join(commonality_path, 'r2_results_brain_body_vs_age.csv'))
r2_decompose = r2_brain_body_vs_age.copy()

r2_decompose['Modality'] = r2_decompose['Modality'].map(modality_names)

for idx, row in r2_decompose.iterrows():
    
    # Calculate variance components
    u_modality = row['R2_combined'] - row['R2_age']
    u_age = row['R2_combined'] - row['R2_modality']
    common = row['R2_modality'] + row['R2_age'] - row['R2_combined']
    
    total = row['R2_combined']

    # Calculate percentages
    perc_modality = round((u_modality / total) * 100, 2) if total != 0 else 0
    perc_age = round((u_age / total) * 100, 2) if total != 0 else 0
    perc_common = round((common / total) * 100, 2) if total != 0 else 0

    # Calculate proportions
    prop_age_exp_by_modality = round((common / (u_age + common)) * 100, 2) if (u_age + common) != 0 else 0
    prop_modality_expl_by_age = round((common / (u_modality + common)) * 100, 2) if (u_modality + common) != 0 else 0

    # Store results
    r2_decompose.at[idx, 'Unique variance: brain/body'] = round(u_modality, 4)
    r2_decompose.at[idx, 'Unique variance: age'] = round(u_age, 4)
    r2_decompose.at[idx, 'Common variance'] = round(common, 4)
    r2_decompose.at[idx, 'Perc unique brain/body'] = perc_modality
    r2_decompose.at[idx, 'Perc unique age'] = perc_age
    r2_decompose.at[idx, 'Perc common'] = perc_common
    r2_decompose.at[idx, 'g-age explained by brain/body'] = round(prop_age_exp_by_modality, 4)
    r2_decompose.at[idx, 'g-brain/body explained by age'] = round(prop_modality_expl_by_age, 4)

    # Print
    print(f"\nAnalysis for {row['Modality']} ({row['Type']}) vs age:")
    print('____________________')
    print('R2 brain/body:', round(row['R2_modality'], 4))
    print('R2 age:', round(row['R2_age'], 4))
    print('R2 combined:', round(row['R2_combined'], 4))
    print('____________________')
    print(f"Unique to brain/body: {round(u_modality, 4):.4f} ({perc_modality:.2f}%)")
    print(f"Unique to age: {round(u_age, 4):.4f} ({perc_age:.2f}%)")
    print(f"Common variance: {round(common, 4):.4f} ({perc_common:.2f}%)")
    print('____________________')
    print(f"Proportion of age variance explained by brain/body: {prop_age_exp_by_modality:.2f}%")
    print(f"Proportion of brain/body variance explained by age: {prop_modality_expl_by_age:.2f}%")
    
# Save
r2_decompose.to_csv(os.path.join(commonality_path, 'commonality_results_decomposed_brain_body_ind_mod_vs_age.csv'), index=False)

r2_decompose.to_excel(
    os.path.join(commonality_path, 'commonality_results_decomposed_brain_body_ind_mod_vs_age.xlsx'),
    index=False,
    engine='openpyxl'
)

# Sort by how much age variance is explained by each modality
r2_decompose.sort_values(by='g-age explained by brain/body', ascending=False).to_excel(
    os.path.join(commonality_path, 'commonality_results_decomposed_brain_body_ind_mod_vs_age_sorted.xlsx'),
    index=False,
    engine='openpyxl'
)

print("\nDecomposition analysis complete. Results saved to:", commonality_path)

/nAnalysis for Immune (body) vs age:
____________________
R2 brain/body: 0.0143
R2 age: 0.1558
R2 combined: 0.1573
____________________
Unique to brain/body: 0.0015 (0.95%)
Unique to age: 0.1430 (90.91%)
Common variance: 0.0128 (8.14%)
____________________
Proportion of age variance explained by brain/body: 8.22%
Proportion of brain/body variance explained by age: 89.51%
/nAnalysis for Renal & Hepatic (body) vs age:
____________________
R2 brain/body: 0.046
R2 age: 0.158
R2 combined: 0.1599
____________________
Unique to brain/body: 0.0019 (1.19%)
Unique to age: 0.1139 (71.23%)
Common variance: 0.0441 (27.58%)
____________________
Proportion of age variance explained by brain/body: 27.91%
Proportion of brain/body variance explained by age: 95.87%
/nAnalysis for Metabolic (body) vs age:
____________________
R2 brain/body: 0.0468
R2 age: 0.1495
R2 combined: 0.1514
____________________
Unique to brain/body: 0.0019 (1.25%)
Unique to age: 0.1046 (69.09%)
Common variance: 0.0449 (29.66%)
___

In [ ]:
# Rename modalities in excel
name_mappings = {
    'allmri': '3 brain MRI modalities stacked',
    'dwi': 'Brain dwMRI stacked',
    'smri': 'Brain sMRI stacked',
    'rs': 'Brain rsMRI stacked',
    'body': 'Body physiology and composition stacked',
    'body-comp': 'Body composition stacked',
    'cardiopulmonary': 'Cardiopulmonary stacked',
    'renal-hepatic': 'Renal & Hepatic stacked',
    'lifestyle-envir': 'Lifestyle & Environment',
    'brain-plus-body': 'Brain & body stacked',
    'body-only': 'Body physiology and composition stacked' 
}

for file_suffix in ['', '_sorted']:
    file_path = os.path.join(commonality_path, f'commonality_results_decomposed_brain_body_ind_mod_vs_age{file_suffix}.xlsx')
    
    # Load the Excel file
    df = pd.read_excel(file_path)
    
    # Apply name mappings to the relevant columns
    #df['Modality'] = df['Modality'].replace(name_mappings)

    df = df.rename(columns = {
    'R2_modality':'R2 brain/body',
    'R2_age':'R2 age',
    'R2_combined':'R2 combined',
    })
    
    # Save back to the same file
    df.to_excel(file_path, index=False, engine='openpyxl')

print("Excel files have been updated with the new column names.")

In [ ]:
# Combnie tables for individual and stacked phenotypes
ind = pd.read_excel(os.path.join(commonality_path, 'commonality_results_decomposed_brain_body_ind_mod_vs_age_sorted.xlsx'))
stack = pd.read_excel(os.path.join(commonality_path, f'commonality_results_decomposed_all_vs_age_sorted.xlsx'))
pd.concat([stack, ind], axis=0).sort_values(by='g-age explained by brain/body', ascending=False).to_excel(os.path.join(commonality_path, f'commonality_results_decompos_brain_body_vs_age_sorted_stack_ind_combined.xlsx'), index=False, engine='openpyxl')

ind = pd.read_excel(os.path.join(commonality_path, 'commonality_results_decomposed_brain_body_ind_mod_vs_age_sorted.xlsx'))
stack = pd.read_excel(os.path.join(commonality_path, f'commonality_results_decomposed_all_vs_age_sorted.xlsx'))
pd.concat([stack, ind], axis=0).sort_values(by='g-age explained by brain/body', ascending=False).to_excel(os.path.join(commonality_path, f'commonality_results_decompos_brain_body_vs_age_sorted_stack_ind_combined_SORTED_BY_BRAIN.xlsx'), index=False, engine='openpyxl')

## 3. g-age explained by individual body and brain phenotypes and composite markers - stratified by sex

In [ ]:
# Configuration
warnings.filterwarnings("ignore", category=DeprecationWarning)
folds = range(0, 5)
base_path = '/UK_BB/brainbody'
stacking_path = os.path.join(base_path, 'stacking')
commonality_path = os.path.join(base_path, 'commonality')

# Load demographic data
demo_age_sex = demo[['eid', 'Age', 'Sex']]

# Define modality groups
modality_groups = {
    'brain': {
        'submodalities': ['allmri', 'dwi', 'rs', 'smri'],
        'pred_pattern': '{submod}_target_pred_2nd_level_rf_test_fold_{fold}.csv',
        'obs_pattern': '{submod}/features_test_level1_outer/features_test_level1_outer_g_matched_fold_{fold}.csv'
    },
    'body': {
        'submodalities': ['body'],
        'pred_pattern': '{submod}_target_pred_2nd_level_{alg}_outer_test_fold_{fold}.csv',
        'obs_pattern': 'features_test_level1_stacked_outer/features_test_level1_outer_g_matched_fold_{fold}.csv'
    },
    'combined': {
        'submodalities': ['brain-body'],
        'pred_pattern': '{submod}_target_pred_2nd_level_{alg}_outer_test_fold_{fold}.csv',
        'obs_pattern': 'features_test_level1_stacked_outer/features_test_level1_outer_g_matched_fold_{fold}.csv'
    }
}

In [ ]:
# Define analysis functions
def compute_r2(data, group, modality, modality_type):
    """Calculate R2 values for a given dataset (modified for age analysis)"""
    if len(data) == 0:
        return None
        
    r2_mod = commonality_analysis(data[f'g_pred_{modality}_test'], data['g_obs_test'])
    r2_age = commonality_analysis(data['Age'], data['g_obs_test'])
    r2_combined = commonality_analysis(
        pd.concat([data[f'g_pred_{modality}_test'], data['Age']], axis=1),
        data['g_obs_test']
    )
    
    return {
        'Modality': modality,
        'Type': modality_type,
        'Sex': group,
        'N': len(data),
        'R2_modality': r2_mod,
        'R2_age': r2_age,
        'R2_combined': r2_combined
    }

def decompose_variance(row):
    """Calculate variance components for a single row (modified for age analysis)"""
    u_modality = row['R2_combined'] - row['R2_age']
    u_age = row['R2_combined'] - row['R2_modality']
    common = row['R2_modality'] + row['R2_age'] - row['R2_combined']
    
    return {
        'Unique variance: brain/body': round(u_modality, 4),
        'Unique variance: age': round(u_age, 4),
        'Common variance': round(common, 4),
        'Perc unique brain/body': round((u_modality/row['R2_combined'])*100, 2) if row['R2_combined'] else 0,
        'Perc unique age': round((u_age/row['R2_combined'])*100, 2) if row['R2_combined'] else 0,
        'Perc common': round((common/row['R2_combined'])*100, 2) if row['R2_combined'] else 0,
        'g-age explained by brain/body': round((common/(u_age+common))*100, 2) if (u_age+common) else 0,
        'g-brain/body explained by age': round((common/(u_modality+common))*100, 2) if (u_modality+common) else 0
    }

def process_modality_combinations(modality, modality_type, is_individual=False):
    """Process data for one modality with age analysis"""
    df_concat = []
    
    for fold in folds:
        try:
            print(f'\n{"#"*15} FOLD {fold} - {modality} {"#"*15}')
            
            # Get predictions path - different paths for stacked vs individual
            if is_individual:
                # For individual modalities
                data_path = os.path.join(base_path, 'lifestyle-envir-body' if modality in modalities_body else 'brain')
                pred_path = os.path.join(
                    data_path, f'folds/fold_{fold}/g_pred/{modality}_g_pred_XGB_test_with_id_fold_{fold}.csv')
                pred = pd.read_csv(pred_path).rename(
                    columns={f'g_pred_test_{modality}': f'g_pred_{modality}_test'})
            else:
                # For stacked models
                if modality_type == 'brain':
                    pred_path = os.path.join(
                        stacking_path, 'brain', modality, 'folds', f'fold_{fold}', 'g_pred',
                        modality_groups[modality_type]['pred_pattern'].format(submod=modality, fold=fold))
                elif modality_type == 'body':
                    if modality == 'body':
                        pred_path = os.path.join(
                            stacking_path, 'body', 'folds', f'fold_{fold}', 'g_pred',
                            'body_target_pred_2nd_level_0_outer_test_fold_{fold}.csv'.format(fold=fold))
                    else:
                        pred_path = os.path.join(
                            stacking_path, 'body', modality, 'folds', f'fold_{fold}', 'g_pred',
                            modality_groups[modality_type]['pred_pattern'].format(submod=modality, alg='rf', fold=fold))
                else:  # combined
                    pred_path = os.path.join(
                        stacking_path, 'brain-body', 'folds', f'fold_{fold}', 'g_pred',
                        modality_groups[modality_type]['pred_pattern'].format(submod=modality, alg='0', fold=fold))
                
                pred = pd.read_csv(pred_path).rename(
                    columns={'g_pred_stack_test': f'g_pred_{modality}_test'})
            
            print('\nPredictions:')
            print('Shape:', pred.shape)
            print('Columns:', pred.columns.tolist())
            print('NA counts:\n', pred.isna().sum())
            
            # Get observations path
            if is_individual:
                # For individual modalities, use brain observations as reference
                obs_path = os.path.join(base_path, f'stacking/brain/allmri/features_test_level1_outer/features_test_level1_outer_g_matched_fold_{fold}.csv')
            else:
                if modality_type == 'brain':
                    obs_path = os.path.join(
                        stacking_path, 'brain',
                        modality_groups[modality_type]['obs_pattern'].format(submod=modality, fold=fold))
                else:  # body or combined
                    obs_path = os.path.join(
                        stacking_path, 'body' if modality_type == 'body' else 'brain-body',
                        modality_groups[modality_type]['obs_pattern'].format(fold=fold))
            
            g_obs = pd.read_csv(obs_path)[['eid', 'g']].rename(columns={'g': 'g_obs_test'})
            
            print('\nObservations:')
            print('Shape:', g_obs.shape)
            print('Columns:', g_obs.columns.tolist())
            print('NA counts:\n', g_obs.isna().sum())
            
            # Merge all data including age
            all_g = (pred.merge(g_obs, on='eid')
                     .merge(demo_age_sex, on='eid')
                     .drop(columns=['eid']))
            
            print('\nMerged data:')
            print('Shape:', all_g.shape)
            print('Columns:', all_g.columns.tolist())
            print('NA counts:\n', all_g.isna().sum())
            
            df_concat.append(all_g)
            
        except FileNotFoundError as e:
            print(f"File not found: {e}")
            continue
    
    if not df_concat:
        return None
    
    # Combine all folds
    all_g = pd.concat(df_concat, axis=0, ignore_index=True)
    output_name = f'g_obs_pred_{modality}_vs_age'
    all_g.to_csv(os.path.join(commonality_path, f'{output_name}.csv'), index=False)
    
    print(f'\n{"#"*15} FINAL MERGED DATA - {modality} {"#"*15}')
    print('Final shape:', all_g.shape)
    print('Final columns:', all_g.columns.tolist())
    print('Final NA counts:\n', all_g.isna().sum())
    
    # Process results by sex group
    results = []
    
    for sex_code, group_name in [(1, 'Male'), (0, 'Female')]:
        sex_data = all_g[all_g['Sex'] == sex_code].copy()
        if len(sex_data) == 0:
            continue
            
        # Calculate R2 metrics using modified compute_r2
        result = compute_r2(sex_data, group_name, modality, modality_type)
        
        if result:
            # Add decomposed variance
            result.update(decompose_variance(result))
            results.append(result)
    
    return results

In [ ]:
# Commonality analysis
all_results = []

# Process stacked models
for group_name, group_config in modality_groups.items():
    for submod in group_config['submodalities']:
        print(f"\nProcessing STACKED {submod} ({group_name}) with age")
        results = process_modality_combinations(submod, group_name, is_individual=False)
        if results:
            all_results.extend(results)
            print(f"Completed stacked {submod}")

# Process individual phenotypes
for modality in modalities_brain_body:
    modality_type = 'body' if modality in modalities_body else 'brain'
    print(f"\nProcessing INDIVIDUAL {modality} ({modality_type}) with age")
    results = process_modality_combinations(modality, modality_type, is_individual=True)
    if results:
        all_results.extend(results)
        print(f"Completed individual {modality}")

# Convert to DataFrame and map modality names
if all_results:
    results_df = pd.DataFrame(all_results)
    results_df['Modality'] = results_df['Modality'].map(modality_names)
    
    # Save outputs
    results_df.to_csv(
        os.path.join(commonality_path, 'commonality_results_decomposed_age_vs_brain_body_sex_strat.csv'), 
        index=False
    )
    
    results_df.to_excel(
        os.path.join(commonality_path, 'commonality_results_decomposed_age_vs_brain_body_sex_strat.xlsx'),
        index=False,
        engine='openpyxl'
    )
    
    # Sort by g-age explained by brain/body
    results_sorted = pd.concat([
        results_df[results_df['Sex'] == 'Male']
        .sort_values('g-age explained by brain/body', ascending=False),
        results_df[results_df['Sex'] == 'Female']
        .sort_values('g-age explained by brain/body', ascending=False)
    ])
    
    results_sorted.drop(columns = ['Type']).rename(columns = {
        'R2_modality':'R2 brain/body',
        'R2_age':'R2 age',
        'R2_combined':'R2 combined'
    }).to_excel(
        os.path.join(commonality_path, 'commonality_results_decomposed_age_vs_brain_body_sex_strat_sorted.xlsx'),
        index=False,
        engine='openpyxl'
    )

    print("\nSaved outputs:")
    print("- commonality_results_decomposed_age_vs_brain_body_sex_strat.csv")
    print("- commonality_results_decomposed_age_vs_brain_body_sex_strat.xlsx")
    print("- commonality_results_decomposed_age_vs_brain_body_sex_strat_sorted.xlsx")
    
else:
    print("\nNo results were generated - check input files")

print("\nAnalysis complete!")

In [ ]:
# Reorganize excel table for age analysis
sorted_path = os.path.join(commonality_path, 'commonality_results_decomposed_age_vs_brain_body_sex_strat_sorted.xlsx')
df_sorted = pd.read_excel(sorted_path)

# Create a new DataFrame with male-female pairs for each modality
paired_results = []

# Get unique modalities (since we're comparing each modality to age)
modalities = df_sorted['Modality'].unique()

for modality in modalities:
    # Get male and female results for this modality
    male = df_sorted[(df_sorted['Modality'] == modality) &
                    (df_sorted['Sex'] == 'Male')]
    
    female = df_sorted[(df_sorted['Modality'] == modality) &
                      (df_sorted['Sex'] == 'Female')]
    
    # Add male then female if both exist
    if not male.empty:
        paired_results.append(male.iloc[0])
    if not female.empty:
        paired_results.append(female.iloc[0])

# Create the new paired DataFrame
df_paired = pd.DataFrame(paired_results)

# Save the new organized version
paired_output_path = os.path.join(commonality_path, 'commonality_results_age_vs_brain_body_paired_male_female.xlsx')
df_paired.to_excel(paired_output_path, index=False, engine='openpyxl')

print(f"\nCreated new paired file at: {paired_output_path}")
print(f"Total modalities processed: {len(modalities)}")
print(f"Final paired entries: {len(df_paired)}")
print("\nFile structure:")
print("- Each modality appears consecutively")
print("- Male result first, followed by female result")
print("- Maintains all original columns and sorting within sex groups")

## 4. g-age explained by composite body plus - 3-var commonality

- U (i) = R2_ijk – R2_jk
- U (j) = R2_ijk – R2_ik
- U (k) = R2_ijk – R2_ij

- C (ij) = R2_ik + R2_jk – R2_k – R2_ijk
- C (ik) = R2_ij + R2_jk – R2_j – R2_ijk
- C (jk) = R2_ij + R2_ik – R2_i – R2_ijk
- C (ijk) = R2_i + R2_j + R2_k – R2_ij – R2_ik – R2_jk + R2_ijk

- U (body) = R2_body_brain_age – R2_brain_age
- U (brain) = R2_body_brain_age – R2_body_age
- U (age) = R2_body_brain_age – R2_body_brain

- C (body_brain) = R2_body_age + R2_brain_age – R2_age – R2_body_brain_age
- C (body_age) = R2_body_brain + R2_brain_age – R2_brain – R2_body_brain_age
- C (body_age) = R2_body_brain + R2_body_age – R2_body – R2_body_brain_age
- C (body_brain_age) = R2_body + R2_brain + R2_age – R2_body_brain – R2_body_age – R2_brain_age + R2_body_brain_age

In [ ]:
# Commonality analysis
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Configuration
folds = range(0, 5)
stacking_path = '/UK_BB/brainbody/stacking'
commonality_path = '/UK_BB/brainbody/commonality'
os.makedirs(commonality_path, exist_ok=True)

# Load demographic data
demo_age = demo[['eid', 'Age']]

# Store concatenated data for each fold
df_concat = []

for fold in folds:
    print(f'\n############## FOLD {fold} ##############')
    
    # Get predictions paths
    pred_path_brain = os.path.join(
        stacking_path, 'brain', 'allmri', 'folds', f'fold_{fold}', 'g_pred',
        f'allmri_target_pred_2nd_level_rf_test_fold_{fold}.csv')
    
    pred_path_body = os.path.join(
        stacking_path, 'body', 'folds', f'fold_{fold}', 'g_pred',
        f'body_target_pred_2nd_level_0_outer_test_fold_{fold}.csv')
    
    # Load predictions
    pred_brain = pd.read_csv(pred_path_brain).rename(
        columns={'g_pred_stack_test': 'g_pred_brain'})
    
    pred_body = pd.read_csv(pred_path_body).rename(
        columns={'g_pred_stack_test': 'g_pred_body'})
    
    # Get observations paths
    obs_path_brain = os.path.join(
        stacking_path, 'brain',
        rf'allmri/features_test_level1_outer/features_test_level1_outer_g_matched_fold_{fold}.csv')
    
    obs_path_body = os.path.join(
        stacking_path, 'body',
        rf'features_test_level1_stacked_outer/features_test_level1_outer_g_matched_fold_{fold}.csv')
    
    g_obs_brain = pd.read_csv(obs_path_brain)[['eid', 'g']].rename(columns={'g': 'g_obs'})
    g_obs_body = pd.read_csv(obs_path_body)[['eid', 'g']].rename(columns={'g': 'g_obs'})
    
    # Restrict to subjects present in both datasets
    g_obs = g_obs_brain.merge(g_obs_body, on='eid', suffixes=('_brain', '_body'))
    
    # Ensure g values match for all common subjects
    if not (g_obs['g_obs_brain'] == g_obs['g_obs_body']).all():
        raise ValueError(f"Mismatch in g_obs values in fold {fold}")
    
    # Keep a single g_obs column
    g_obs = g_obs[['eid', 'g_obs_brain']].rename(columns={'g_obs_brain': 'g_obs'})

    # Merge all data including age
    all_g = (pred_brain.merge(pred_body, on='eid')
             .merge(g_obs, on='eid')
             .merge(demo_age, on='eid')
             .drop(columns=['eid']))
    
    print('Merged data for this fold:')
    print('Shape:', all_g.shape)
    print('Columns:', all_g.columns.tolist())
    print('NA counts:', all_g.isna().sum().sum())
    
    df_concat.append(all_g)

# Check if there are any data
if not df_concat:
    print("No data loaded! Check file paths.")
    exit()

# Concatenate all folds
all_g_body_brain_age = pd.concat(df_concat, axis=0, ignore_index=True)
all_g_body_brain_age.to_csv(os.path.join(commonality_path, f'g_obs_pred_brain_plus_body_with_age.csv'), index=False)

print(f'\n############## FINAL MERGED DATA ##############')
print('Final shape:', all_g_body_brain_age.shape)
print('Final columns:', all_g_body_brain_age.columns.tolist())
print('NA counts:\n', all_g_body_brain_age.isna().sum())

# Calculate R2 values for 3-variable commonality analysis
print('\n############## CALCULATING R2 VALUES ##############')

r2_body = commonality_analysis(all_g_body_brain_age['g_pred_body'], all_g_body_brain_age['g_obs'])
r2_brain = commonality_analysis(all_g_body_brain_age['g_pred_brain'], all_g_body_brain_age['g_obs'])
r2_age = commonality_analysis(all_g_body_brain_age['Age'], all_g_body_brain_age['g_obs'])
r2_body_brain = commonality_analysis(
    pd.concat([all_g_body_brain_age['g_pred_brain'], all_g_body_brain_age['g_pred_body']], axis=1), 
    all_g_body_brain_age['g_obs']
)
r2_body_age = commonality_analysis(
    pd.concat([all_g_body_brain_age['g_pred_body'], all_g_body_brain_age['Age']], axis=1), 
    all_g_body_brain_age['g_obs']
)
r2_brain_age = commonality_analysis(
    pd.concat([all_g_body_brain_age['g_pred_brain'], all_g_body_brain_age['Age']], axis=1), 
    all_g_body_brain_age['g_obs']
)
r2_body_brain_age = commonality_analysis(
    pd.concat([all_g_body_brain_age['g_pred_brain'], all_g_body_brain_age['g_pred_body'], all_g_body_brain_age['Age']], axis=1), 
    all_g_body_brain_age['g_obs']
)

print('\nR2 RESULTS:')
print('____________________')
print(f'R2 Body: {r2_body:.4f}')
print(f'R2 Brain: {r2_brain:.4f}')
print(f'R2 Age: {r2_age:.4f}')
print(f'R2 Body + Brain: {r2_body_brain:.4f}')
print(f'R2 Body + Age: {r2_body_age:.4f}')
print(f'R2 Brain + Age: {r2_brain_age:.4f}')
print(f'R2 Body + Brain + Age: {r2_body_brain_age:.4f}')

# Calculate commonality components
print('\n############## COMMONALITY COMPONENTS ##############')

# Unique variances (U)
u_body = r2_body_brain_age - r2_brain_age
u_brain = r2_body_brain_age - r2_body_age
u_age = r2_body_brain_age - r2_body_brain

print("Unique variance for Body:", round((u_body),4))
print("Unique variance for Brain:", round((u_brain),4))
print("Unique variance for Age:", round((u_age),4))

# Common variances (C)
c_body_brain = r2_body_age + r2_brain_age - r2_age - r2_body_brain_age
c_body_age = r2_body_brain + r2_brain_age - r2_brain - r2_body_brain_age
c_brain_age = r2_body_brain + r2_body_age - r2_body - r2_body_brain_age
c_body_brain_age = r2_body + r2_brain + r2_age - r2_body_brain - r2_body_age - r2_brain_age + r2_body_brain_age

age_total = u_age + c_body_age + c_body_brain_age + c_brain_age

prop_age_expl_by_body_only = (c_body_age + c_body_brain_age) / (age_total)
prop_age_expl_by_brain_only = (c_brain_age + c_body_brain_age) / (age_total)
prop_age_expl_by_brain_OR_body = (c_body_age + c_brain_age + c_body_brain_age) / (age_total)
prop_age_expl_by_brain_AND_body = (c_body_brain_age) / (age_total)

print("Common variance for Body & Brain:", round((c_body_brain),4))
print("Common variance for Body & Age:", round((c_body_age),4))
print("Common variance for Brain & Age:", round((c_brain_age),4))
print("Common variance for Body, Brain & Age:", round((c_body_brain_age),4))


# Build a single-row decomposition table
decomp = pd.DataFrame([{
    # R2
    'R2 Body': round(r2_body, 2),
    'R2 Brain': round(r2_brain, 4),
    'R2 Age': round(r2_age, 4),
    'R2 Body + Brain': round(r2_body_brain, 2),
    'R2 Body + Age': round(r2_body_age, 4),
    'R2 Brain + Age': round(r2_brain_age, 4),
    'R2 Body + Brain + Age': round(r2_body_brain_age, 4),
    # Raw variances
    'Unique: Body': round(u_body, 4),
    'Unique: Brain': round(u_brain, 4),
    'Unique: Age': round(u_age, 4),
    'Common: Body & Brain': round(c_body_brain, 4),
    'Common: Body & Age': round(c_body_age, 4),
    'Common: Brain & Age': round(c_brain_age, 4),
    'Common: All Three': round(c_body_brain_age, 4),

    # Percentages
    'Perc unique body': round(u_body / r2_body_brain_age * 100, 2),
    'Perc unique brain': round(u_brain / r2_body_brain_age * 100, 2),
    'Perc unique age': round(u_age / r2_body_brain_age * 100, 2),
    'Perc common body-brain': round(c_body_brain / r2_body_brain_age * 100, 2),
    'Perc common body-age': round(c_body_age / r2_body_brain_age * 100, 2),
    'Perc common brain-age': round(c_brain_age / r2_body_brain_age * 100, 2),
    'Perc common all three': round(c_body_brain_age / r2_body_brain_age * 100, 2),

    # Additional interpretable quantities
    'g-age explained by body': round(prop_age_expl_by_body_only, 4)*100,
    'g-age explained by brain': round(prop_age_expl_by_brain_only, 4)*100,
    'g-age explained by brain / body': round(prop_age_expl_by_brain_OR_body, 4)*100,
    'g-age explained by brain & body': round(prop_age_expl_by_brain_AND_body, 4)*100,

    # Total R2
    'Total R2 (Body+Brain+Age)': round(r2_body_brain_age, 4)}])


# Save results
decomp.to_csv(
    os.path.join(commonality_path, 'commonality_results_body_plus_brain_age_3v.csv'),
    index=False
)

decomp.to_excel(
    os.path.join(commonality_path, 'commonality_results_body_plus_brain_age_3v.xlsx'),
    index=False,
    engine='openpyxl'
)

print("\nFinal Results:")
print(decomp)

## Combine tables: body and brain

Combine composite body and body phenotypes in one table

In [ ]:
# Combnie individual modalities and stacked: brain/body
ind = pd.read_excel(os.path.join(commonality_path, 'commonality_results_decompos_brain_vs_individual_body_sorted.xlsx'))
stack = pd.read_excel(os.path.join(commonality_path, f'commonality_results_decompos_brain_vs_body_stacked_sorted.xlsx'))
pd.concat([stack, ind], axis=0).sort_values(by='g-body explained by brain', ascending=False).to_excel(os.path.join(commonality_path, f'commonality_results_decompos_brain_vs_body_sorted_stack_ind_combined.xlsx'), index=False, engine='openpyxl')

Rename body/brain

In [ ]:
# Define renaming function
def process_modality_name(name):
    if pd.isna(name):
        return name
    
    # Handle specific replacements
    name = name.replace('Bone densitometry (of heel)', 'Bone Densitometry of Heel')
    name = name.replace('3 brain MRI modalities stacked', '3 Brain MRI Modalities Stacked')
    name = name.replace('stacked', 'Stacked')
    name = name.replace('Body physiology and composition Stacked', 'Body Physiology and Composition Stacked')
    name = name.replace('Bone size, mineral and density by DXA', 'Bone Size, Mineral and Density by DXA')
    name = name.replace('composition', 'Composition')
    
    
    # Capitalization - keep existing capitalization for all words
    #words = name.split()
    #processed_words = [word.capitalize() for word in words]
    
    #result = ' '.join(processed_words)
    return name

In [ ]:
# Correct phenotypes names and make them uniform
file_path = os.path.join(commonality_path, 'commonality_results_decompos_brain_vs_body_sorted_stack_ind_combined.xlsx')

df = pd.read_excel(file_path, engine='openpyxl')
# Apply the processing to the Modality column
df['Body modality'] = df['Body modality'].apply(process_modality_name)
df['Brain MRI modality'] = df['Brain MRI modality'].apply(process_modality_name)
#df = df.drop(columns='Type')
df = df[~df['Body modality'].isin(['Body composition stacked', 'Cardiopulmonary Stacked', 'Renal & Hepatic Stacked', 'Body Composition Stacked'])]
#df = df[df['Modality'] != 'Invalid Modality']

df.to_excel(os.path.join(commonality_path, f'commonality_results_decompos_brain_vs_body_sorted_stack_ind_combined_RENAMED.xlsx'), index=False, engine='openpyxl')
#f'commonality_results_decompos_brain_body_vs_age_sorted_stack_ind_combined_SORTED_BY_AGE_RENAMED.xlsx'
# Display the results to verify
print("Processed Modality column values:")
print(df['Brain MRI modality'].unique())
print(df['Body modality'].unique())

Processed Modality column values:
['3 Brain MRI Modalities Stacked' 'Brain dwMRI Stacked'
 'Brain sMRI Stacked' 'Brain rsMRI Stacked']
['Liver MRI' 'ECG at Rest' 'Kidney MRI' 'Immune'
 'Bone Densitometry of Heel' 'Heart MRI' 'Metabolic' 'Pulse Wave Analysis'
 'Carotid Ultrasound' 'Arterial Stiffness' 'Cardiopulmonary'
 'Renal & Hepatic' 'Bone Size, Mineral and Density by DXA'
 'Body Composition by Impedance' 'Body Composition by DXA'
 'Abdominal Organ Composition by MRI' 'Musculoskeletal'
 'Abdominal Composition by MRI' 'Body Physiology and Composition Stacked'
 'Hearing']


Rename age+body/brain

In [ ]:
# Define renaming function
def process_modality_name(name):
    if pd.isna(name):
        return name
    
    # Handle replacements
    name = name.replace('Corr.', 'Correlation').replace('Corr.', 'Correlation')
    name = name.replace('Prob.', 'Probabilistic')
    name = name.replace('(of heel)', 'of Heel')
    
    # Split into words and process each one
    words = name.split()
    processed_words = []
    
    for word in words:
        # Handle specific lowercase terms - but NOT 'iv' here since we need to handle it differently
        if word.lower() in ['and', 'aparc', 'by', 'at', 'ex-vivo', 'ex', 'vivo', 'of', 'heel', 'a2009s', 'aparc.a2009s']:
            processed_words.append(word.lower())
        # Keep specific MRI modalities as-is
        elif word in ['dwMRI', 'rsMRI', 'sMRI', 'MRI']:
            processed_words.append(word)
        # Handle hyphenated words separately to preserve 'iv' vs 'IV'
        elif '-' in word:
            parts = word.split('-')
            processed_parts = []
            for part in parts:
                if part.lower() in ['aparc', 'by', 'at', 'ex', 'vivo', 'of', 'heel', 'a2009s', 'and', 'aparc.a2009s']:
                    processed_parts.append(part.lower())
                elif part in ['dwMRI', 'rsMRI', 'sMRI', 'MRI', 'SIFT2', 'FBC', 'FA', 'TBSS', 'MD', 'MO', 'OD', 'ICVF', 'ISOVF', 'ECG', 'DXA', 'IV', 'I']:
                    processed_parts.append(part)
                elif (part.isupper() or 
                      (any(char.isupper() for char in part) and 
                       (any(char.isdigit() for char in part) or len(part) <= 4))):
                    processed_parts.append(part)
                else:
                    processed_parts.append(part.capitalize())
            processed_words.append('-'.join(processed_parts))
        # Keep acronyms and abbreviations
        elif (word.isupper() or 
              word in ['dwMRI', 'rsMRI', 'sMRI', 'MRI', 'SIFT2', 'FBC', 'FA', 'TBSS', 'MD', 'MO', 'OD', 'ICVF', 'ISOVF', 'ECG', 'DXA', 'IV', 'I'] or
              (any(char.isupper() for char in word) and 
               (any(char.isdigit() for char in word) or len(word) <= 4))):
            processed_words.append(word)
        # Capitalize all other words (regular words)
        else:
            processed_words.append(word.capitalize())
    
    result = ' '.join(processed_words)
    result = result.replace('Corr.', 'Correlation')
    return result

In [ ]:
# Correct phenotypes names and make them uniform
file_path = os.path.join(commonality_path, 'commonality_results_age_vs_brain_body_paired_male_female.xlsx')
#'commonality_results_decompos_brain_body_vs_age_sorted_stack_ind_combined_SORTED_BY_BRAIN.xlsx')

df = pd.read_excel(file_path, engine='openpyxl')
# Apply the processing to the Modality column
df['Modality'] = df['Modality'].apply(process_modality_name)
#df = df.drop(columns='Type')
df = df[~df['Modality'].isin(['Body Composition Stacked', 'Cardiopulmonary Stacked', 'Renal & Hepatic Stacked'])]
#df = df[df['Modality'] != 'Invalid Modality']

df.to_excel(os.path.join(commonality_path, f'commonality_results_age_vs_brain_body_paired_male_female_RENAMED.xlsx'), index=False, engine='openpyxl')
#f'commonality_results_decompos_brain_body_vs_age_sorted_stack_ind_combined_SORTED_BY_AGE_RENAMED.xlsx'
# Display the results to verify
print("Processed Modality column values:")
print(df['Modality'].unique())

Processed Modality column values:
['Brain dwMRI Stacked' 'Body Physiology Stacked'
 '3 Brain MRI Modalities Stacked' 'Schaefer7n500p-IV Streamline Count'
 'Schaefer7n500p-IV SIFT2 FBC' 'Brain & Body Stacked'
 'Glasser-IV Streamline Count' 'Glasser-IV SIFT2 FBC'
 'Schaefer7n500p-IV FA' 'Schaefer7n500p-IV Length'
 'Body Composition by Impedance' 'Glasser-IV Length' 'Glasser-IV FA'
 'aparc.a2009s-I Streamline Count' 'Glasser-I Streamline Count'
 'Glasser-I SIFT2 FBC' 'Glasser-I FA' 'Schaefer7n200p-I SIFT2 FBC'
 'aparc.a2009s-I SIFT2 FBC' 'aparc.a2009s-I FA'
 'Schaefer7n200p-I Streamline Count' 'aparc.a2009s-I Length'
 'aparc-I Streamline Count' 'Schaefer7n200p-I FA' 'aparc-I SIFT2 FBC'
 'Glasser-I Length' 'aparc-I Length' 'Schaefer7n200p-I Length'
 'aparc-I FA' 'Brain sMRI Stacked' 'ASEG Volume' 'ASEG Mean Intensity'
 'Regional Grey Matter Volumes (FSL FAST)'
 'Bone Size, Mineral and Density by DXA' 'Brain rsMRI Stacked'
 'Abdominal Composition by MRI' 'L2 TBSS'
 'Subcortical Volumetric S